<a href="https://colab.research.google.com/github/True-star-580/AI/blob/main/v22_with_updated__SUBMISSION_IVORY_COAST_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

paulnkamau_ivorycoast_byte_sizedagric_path = kagglehub.dataset_download('paulnkamau/ivorycoast-byte-sizedagric')
paulnkamau_cleaned_frm_v1_v21_path = kagglehub.dataset_download('paulnkamau/cleaned-frm-v1-v21')

print('Data source import complete.')


100%|██████████| 590M/590M [00:04<00:00, 134MB/s]

Extracting files...


100%|██████████| 4.95M/4.95M [00:00<00:00, 95.4MB/s]

Extracting files...
Data source import complete.


In [19]:
train = pd.read_csv(os.path.join(paulnkamau_cleaned_frm_v1_v21_path, 'train_cleaned.csv'))
test = pd.read_csv(os.path.join(paulnkamau_cleaned_frm_v1_v21_path, 'test_cleaned.csv'))

print("Train and test CSVs loaded successfully.")

MAX_TILE_SIZE = 326 # Keep this definition

Train and test CSVs loaded successfully.


In [7]:
# !pip install autogluon -q

In [8]:
!pip install rasterio geopandas -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 130.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 94.0 MB/s eta 0:00:00


In [9]:
import os
from pathlib import Path
import shutil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import gc

In [20]:
!pip install tqdm plotly -q

In [12]:
!pip install -U scikit-image -q


In [13]:
# disti=pd.crosstab(train_jan['month'], train_jan['class'])
# disti

In [14]:
from skimage.feature import graycomatrix, graycoprops


In [15]:
import gc
gc.collect()

8

-----

------
-------

In [22]:

# --- Modified process_and_return_tiles to return processed DataFrame subset ---
import os
import rasterio
import numpy as np
import pandas as pd
from rasterio.windows import Window

In [23]:
import os
import rasterio
import numpy as np # Need numpy for seterr
import pandas as pd
from rasterio.windows import Window
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px
from sklearn.metrics import classification_report, accuracy_score
from scipy import ndimage
from skimage.feature import graycomatrix, graycoprops
from skimage.filters import gabor
from skimage.measure import shannon_entropy
from skimage.feature import local_binary_pattern
from scipy.ndimage import binary_opening, binary_closing, binary_erosion, binary_dilation
from skimage.filters.rank import entropy # Need import specifically for rank
from skimage.morphology import disk # Need import specifically for disk
from skimage.filters import sobel, prewitt, roberts
from skimage.feature import canny # Canny is in feature
from skimage.filters import gaussian # Sometimes used before edge detection

# Import tqdm for progress bars
from tqdm.auto import tqdm

# Import joblib for parallel processing
from joblib import Parallel, delayed

# import warnings # Not strictly needed if using np.seterr instead

# TILE_SIZE must be defined, e.g., TILE_SIZE = 64 or 128 or MAX_TILE_SIZE if MAX_TILE_SIZE is defined elsewhere.
# Assuming MAX_TILE_SIZE is defined globally or imported.
# Example placeholder if not defined elsewhere:
# MAX_TILE_SIZE = 128 # <--- Define this or ensure it's defined elsewhere
TILE_SIZE = MAX_TILE_SIZE # <--- Define this or ensure it's defined elsewhere

# Define TILE_SIZE if not defined elsewhere for this example
# Remove these lines if MAX_TILE_SIZE/TILE_SIZE are truly defined elsewhere
try:
    TILE_SIZE
except NameError:
    print("TILE_SIZE not defined. Using default 128.")
    TILE_SIZE = 128


# --- Feature Extraction Functions (Keeping these consistent with previous response) ---
# (Added robustness checks like epsilon denominators and np.maximum already)

def extract_ndvi(nir_band, red_band, epsilon=1e-8):
    """ Calculate NDVI from NIR and Red bands """
    # Ensure inputs are float to handle division and potential NaN/Inf
    nir_band = nir_band.astype(np.float32)
    red_band = red_band.astype(np.float32)
    # Avoid division by zero/near-zero
    denominator = nir_band + red_band
    return (nir_band - red_band) / (denominator + epsilon)

def extract_savi(nir_band, red_band, L=0.5, epsilon=1e-8):
    """ Calculate SAVI from NIR and Red bands """
    nir_band = nir_band.astype(np.float32)
    red_band = red_band.astype(np.float32)
    # Avoid division by zero/near-zero
    denominator = nir_band + red_band + L
    return ((nir_band - red_band) * (1 + L)) / (denominator + epsilon)

def extract_evi(nir_band, red_band, blue_band, G=2.5, C1=6, C2=7.5, L=10000, epsilon=1e-8):
    """ Calculate EVI from NIR, Red, and Blue bands """
    nir_band = nir_band.astype(np.float32)
    red_band = red_band.astype(np.float32)
    blue_band = blue_band.astype(np.float32)
    denominator = (nir_band + C1 * red_band - C2 * blue_band + L)
    # Avoid division by zero/near-zero
    return G * ((nir_band - red_band) / (denominator + epsilon))

def extract_s2rep(b4, b5, b6, b7, epsilon=1e-8):
    """Calculate S2REP from Red (B4) and Red Edge (B5, B6, B7) bands."""
    b4 = b4.astype(np.float32)
    b5 = b5.astype(np.float32)
    b6 = b6.astype(np.float32)
    b7 = b7.astype(np.float32)
    numerator = (((b7 + b4) / 2) - b5)
    denominator = (b6 - b5)
    # Handle case where b6 == b5
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    s2rep = 705 + 35 * (numerator / safe_denominator)
    return s2rep

def extract_ccci(b4, b5, b8, epsilon=1e-8):
    """Calculate CCCI from NIR (B8), Red (B4), and Red Edge (B5) bands."""
    b4 = b4.astype(np.float32)
    b5 = b5.astype(np.float32)
    b8 = b8.astype(np.float32)
    ndre_denominator = (b8 + b5)
    ndre = (b8 - b5) / (ndre_denominator + epsilon)
    ndvi_denominator = (b8 + b4)
    ndvi = (b8 - b4) / (ndvi_denominator + epsilon)
    return ndre / (ndvi + epsilon) # Avoid division by zero ndvi

def extract_ndwi(b3, b8, epsilon=1e-8):
    """Calculate NDWI from Green (B3) and NIR (B8) bands."""
    b3 = b3.astype(np.float32)
    b8 = b8.astype(np.float32)
    denominator = (b3 + b8)
    return (b3 - b8) / (denominator + epsilon)

# Original vegetation indices
def extract_ndre(b8, b6, epsilon=1e-8):
    """Normalized Difference Red Edge"""
    b8 = b8.astype(np.float32)
    b6 = b6.astype(np.float32)
    denominator = (b8 + b6)
    return (b8 - b6) / (denominator + epsilon)

def extract_mmsr(b8, b4, epsilon=1e-8):
    """Modified Modified Simple Ratio"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    ratio_denominator = (b4)
    ratio = b8 / (ratio_denominator + epsilon)
    # Added epsilon to sqrt
    return (ratio - 1) / (np.sqrt(np.maximum(ratio, 0.0) + epsilon) + 1) # Added maximum for safety

def extract_gndvi(b8, b3, epsilon=1e-8):
    """Green Normalized Difference Vegetation Index"""
    b8 = b8.astype(np.float32)
    b3 = b3.astype(np.float32)
    denominator = (b8 + b3)
    return (b8 - b3) / (denominator + epsilon)

def extract_evi2(b8, b4, epsilon=1e-8):
    """Enhanced Vegetation Index 2"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b8 + 2.4 * b4 + 1)
    return 2.5 * (b8 - b4) / (denominator + epsilon)

def extract_ngrdi(b3, b4, epsilon=1e-8):
    """Normalized Green Red Difference Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b3 + b4)
    return (b3 - b4) / (denominator + epsilon)

def extract_mndwi(b3, b11, epsilon=1e-8):
    """Modified Normalized Difference Water Index"""
    b3 = b3.astype(np.float32)
    b11 = b11.astype(np.float32)
    denominator = (b3 + b11)
    return (b3 - b11) / (denominator + epsilon)

def extract_osavi(b8, b4, epsilon=1e-8):
    """Optimized Soil Adjusted Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b8 + b4 + 0.16)
    return (b8 - b4) / (denominator + epsilon)

def extract_wdrvi(b8, b4, alpha=0.1, epsilon=1e-8):
    """Wide Dynamic Range Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (alpha * b8 + b4)
    return (alpha * b8 - b4) / (denominator + epsilon)

def extract_tgi(b3, b4, b2):
    """Triangular Greenness Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    return b3 - 0.39 * b4 - 0.61 * b2

def extract_gcvi(b8, b3, epsilon=1e-8):
    """Green Chlorophyll Vegetation Index"""
    b8 = b8.astype(np.float32)
    b3 = b3.astype(np.float32)
    return (b8 / (b3 + epsilon)) - 1

def extract_rgvi(b1, b3, b4, b5, b7, epsilon=1e-8):
    """Red Green Vegetation Index"""
    b1 = b1.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b5 = b5.astype(np.float32)
    b7 = b7.astype(np.float32)
    denominator = (b4 + b5 + b7)
    return 1 - ((b1 + b3) / (denominator + epsilon))

def extract_arvi(b8, b4, b2, gamma=1.0, epsilon=1e-8):
    """Atmospherically Resistant Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    # Using user's simplified formula: (b8 - (2 * b4) + b2) / (b8 + (2 * b4) + b2 + epsilon)
    numerator = b8 - (2 * b4) + b2
    denominator = b8 + (2 * b4) + b2
    return numerator / (denominator + epsilon)


def extract_sipi(b8, b2, b4, epsilon=1e-8):
    """Structure Insensitive Pigment Index"""
    b8 = b8.astype(np.float32)
    b2 = b2.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b8 - b4)
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    return (b8 - b2) / safe_denominator

def extract_rendvi(b6, b5, epsilon=1e-8):
    """Red Edge Normalized Difference Vegetation Index"""
    b6 = b6.astype(np.float32)
    b5 = b5.astype(np.float32)
    denominator = (b6 + b5)
    return (b6 - b5) / (denominator + epsilon)

def extract_mresr(b6, b1, b5, epsilon=1e-8):
    """Modified Red Edge Simple Ratio"""
    b6 = b6.astype(np.float32)
    b1 = b1.astype(np.float32)
    b5 = b5.astype(np.float32)
    denominator = (b5 - b1)
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    return (b6 - b1) / safe_denominator

def extract_ryi(b3, b2, epsilon=1e-8):
    """Red Yellow Index"""
    b3 = b3.astype(np.float32)
    b2 = b2.astype(np.float32)
    return b3 / (b2 + epsilon)

def extract_ndyi(b3, b2, epsilon=1e-8):
    """Normalized Difference Yellowness Index"""
    b3 = b3.astype(np.float32)
    b2 = b2.astype(np.float32)
    denominator = (b3 + b2)
    return (b3 - b2) / (denominator + epsilon)

def extract_dyi(b3, b2):
    """Difference Yellowness Index"""
    b3 = b3.astype(np.float32)
    b2 = b2.astype(np.float32)
    return b3 - b2

def extract_aci(b8, b4, b3): # Removed epsilon from formula, it's a product
    """Ashburn Chlorophyll Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    b3 = b3.astype(np.float32)
    # User's formula: b8 * (b4 + b3)
    return b8 * (b4 + b3)

def extract_cvi(b8, b3, b4, epsilon=1e-8):
    """Chlorophyll Vegetation Index"""
    b8 = b8.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    return (b8 / (b3 + epsilon)) * (b4 / (b3 + epsilon))

def extract_avi(b8, b4):
    """Advanced Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b8 * (1 - b4) * (b8 - b4)

def extract_si(b2, b3, b4):
    """Shadow Index"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    return (1 - b2) * (1 - b3) * (1 - b4)

def extract_bsi(b11, b4, b8, b2, epsilon=1e-8):
    """Bare Soil Index"""
    b11 = b11.astype(np.float32)
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    b2 = b2.astype(np.float32)
    denominator = ((b11 + b4) + (b8 + b2))
    return ((b11 + b4) - (b8 + b2)) / (denominator + epsilon)

def extract_mtci(b6, b5, b4, epsilon=1e-8):
    """MERIS Terrestrial Chlorophyll Index"""
    b6 = b6.astype(np.float32)
    b5 = b5.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b5 - b4)
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    return (b6 - b5) / safe_denominator

def extract_npcri(b4, b2, epsilon=1e-8):
    """Normalized Pigment Chlorophyll Ratio Index"""
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    denominator = (b4 + b2)
    return (b4 - b2) / (denominator + epsilon)

def extract_mcari(b5, b4, b3, epsilon=1e-8):
    """Modified Chlorophyll Absorption Ratio Index"""
    b5 = b5.astype(np.float32)
    b4 = b4.astype(np.float32)
    b3 = b3.astype(np.float32)
    denominator_term = (b4)
    return ((b5 - b4) - 0.2 * (b5 - b3)) * (b5 / (denominator_term + epsilon))

def extract_tcari(b5, b4, b3, epsilon=1e-8):
    """Transformed Chlorophyll Absorption Ratio Index"""
    b5 = b5.astype(np.float32)
    b4 = b4.astype(np.float32)
    b3 = b3.astype(np.float32)
    denominator_term = (b4)
    return 3 * ((b5 - b4) - 0.2 * (b5 - b3) * (b5 / (denominator_term + epsilon)))

def extract_pvi(b8, b4):
    """Perpendicular Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    return (b8 - 0.3 * b4 - 0.5) / np.sqrt(1 + 0.3**2)

def extract_bai(b4, b8, epsilon=1e-8):
    """Burn Area Index"""
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    denominator_arg = (0.1 - b4)**2 + (0.06 - b8)**2
    # Avoid division by zero/near-zero. Also handle potential negative values if inputs are strange.
    denominator = np.maximum(denominator_arg, epsilon) # Ensure denominator is positive and non-zero
    return 1 / denominator


def extract_mtvi2(b8, b3, b4):
    """Modified Triangular Vegetation Index 2"""
    b8_safe = np.maximum(b8.astype(np.float32), 1e-8)
    b3_safe = np.maximum(b3.astype(np.float32), 1e-8)
    b4_safe = np.maximum(b4.astype(np.float32), 1e-8)

    sqrt_b4 = np.sqrt(b4_safe)
    discriminant_arg = (2 * b8_safe + 1)**2 - (6 * b8_safe - 5 * sqrt_b4) - 0.5
    sqrt_discriminant = np.sqrt(np.maximum(discriminant_arg, 0))

    mtvi2 = 1.5 * (1.2 * (b8_safe - b3_safe) - 2.5 * (b4_safe - b3_safe)) * sqrt_discriminant
    # Set result to NaN where discriminant was negative (sqrt(negative) is not real)
    mtvi2 = np.where(discriminant_arg < 0, np.nan, mtvi2)
    return mtvi2

def extract_ndsi(b3, b11, epsilon=1e-8):
    """Normalized Difference Snow Index"""
    b3 = b3.astype(np.float32)
    b11 = b11.astype(np.float32)
    denominator = (b3 + b11)
    return (b3 - b11) / (denominator + epsilon)

def extract_mrendvi(b6, b5, b1, epsilon=1e-8):
    """Modified Red Edge Normalized Difference Vegetation Index"""
    b6 = b6.astype(np.float32)
    b5 = b5.astype(np.float32)
    b1 = b1.astype(np.float32)
    denominator = (b6 + b5 - 2 * b1)
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    return (b6 - b5) / safe_denominator

def extract_ndvi_re(b8, b5, epsilon=1e-8):
    """NDVI Red Edge"""
    b8 = b8.astype(np.float32)
    b5 = b5.astype(np.float32)
    denominator = (b8 + b5)
    return (b8 - b5) / (denominator + epsilon)

def extract_ci_re(b8, b5, epsilon=1e-8):
    """Chlorophyll Index Red Edge"""
    b8 = b8.astype(np.float32)
    b5 = b5.astype(np.float32)
    return (b8 / (b5 + epsilon)) - 1

def extract_ndmi(b8, b11, epsilon=1e-8):
    """Normalized Difference Moisture Index"""
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    denominator = (b8 + b11)
    return (b8 - b11) / (denominator + epsilon)

def extract_tndvi(b8, b4, epsilon=1e-8):
    """Transformed NDVI"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    ndvi_denominator = (b8 + b4)
    ndvi = (b8 - b4) / (ndvi_denominator + epsilon)
    arg = ndvi + 0.5
    # Ensure sqrt argument is non-negative
    tndvi = np.sqrt(np.maximum(arg, 0))
    # Set result to NaN where argument was negative
    tndvi = np.where(arg < 0, np.nan, tndvi)
    return tndvi

def extract_vdvi(b3, b4, b2, epsilon=1e-8):
    """Visible Difference Vegetation Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    denominator = (2 * b3 + b4 + b2)
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    return (2 * b3 - b4 - b2) / safe_denominator

def extract_nbr(b8, b11, epsilon=1e-8):
    """Normalized Burn Ratio"""
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    denominator = (b8 + b11)
    return (b8 - b11) / (denominator + epsilon)

def extract_tvi(b6, b3, b4):
    """Triangular Vegetation Index"""
    b6 = b6.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    return (120 * (b6 - b3) - 200 * (b4 - b3)) / 2

def extract_exg(b3, b4, b2):
    """Excess Green Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    return 2 * b3 - b4 - b2

def extract_psri(b4, b2, b6, epsilon=1e-8):
    """Plant Senescence Reflectance Index"""
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    b6 = b6.astype(np.float32)
    # Handle division by near-zero b6
    return (b4 - b2) / (b6 + epsilon)

def extract_rdvi(b8, b4, epsilon=1e-8):
    """Renormalized Difference Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    arg = b8 + b4
    # Ensure sqrt argument is non-negative
    sqrt_sum = np.sqrt(np.maximum(arg, 0))
    safe_sqrt_sum = np.where(np.abs(sqrt_sum) < epsilon, epsilon * np.sign(sqrt_sum + epsilon), sqrt_sum)
    # Handle division by near-zero or zero sqrt_sum
    rdvi = np.zeros_like(b8) # Default to zero
    # Only calculate where safe_sqrt_sum is non-zero
    valid_mask = safe_sqrt_sum != 0
    rdvi[valid_mask] = (b8[valid_mask] - b4[valid_mask]) / safe_sqrt_sum[valid_mask]
    # Set result to NaN where arg was negative initially
    rdvi = np.where(arg < 0, np.nan, rdvi)
    return rdvi

# Ratio indices
def extract_ratio_b1_b3(b1, b3, epsilon=1e-8):
    b1 = b1.astype(np.float32)
    b3 = b3.astype(np.float32)
    return b1 / (b3 + epsilon)

def extract_ratio_b1_b5(b1, b5, epsilon=1e-8):
    b1 = b1.astype(np.float32)
    b5 = b5.astype(np.float32)
    return b1 / (b5 + epsilon)

def extract_ratio_b11_b12(b11, b12, epsilon=1e-8):
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return b11 / (b12 + epsilon)

def extract_ratio_b5_b4(b5, b4, epsilon=1e-8):
    b5 = b5.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b5 / (b4 + epsilon)

def extract_ratio_b7_b5(b7, b5, epsilon=1e-8):
    b7 = b7.astype(np.float32)
    b5 = b5.astype(np.float32)
    return b7 / (b5 + epsilon)

def extract_ratio_b7_b6(b7, b6, epsilon=1e-8):
    b7 = b7.astype(np.float32)
    b6 = b6.astype(np.float32)
    return b7 / (b6 + epsilon)

def extract_ratio_b8_b4(b8, b4, epsilon=1e-8):
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b8 / (b4 + epsilon)

# Additional advanced indices
def extract_lai(b8, b4, epsilon=1e-8):
    """Leaf Area Index approximation using NDVI"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    ndvi_denominator = (b8 + b4)
    ndvi = (b8 - b4) / (ndvi_denominator + epsilon)
    ndvi_clipped = np.clip(ndvi, -1.0, 0.689) # Clip to range
    numerator = 0.69 - ndvi_clipped
    denominator = 0.59 + ndvi_clipped
    # Avoid division by zero/near-zero and ensure log argument is positive
    ratio = np.maximum(numerator / (denominator + epsilon), epsilon)
    log_ratio = np.log(ratio)
    # Avoid division by zero
    safe_divisor = -0.91 # Constant
    lai = log_ratio / (safe_divisor + epsilon if np.abs(safe_divisor) < epsilon else safe_divisor) # Added epsilon for safety if divisor was variable

    # Ensure result is finite (log(<=0) is inf/nan)
    lai = np.where(np.isfinite(lai), lai, np.nan)
    return lai

def extract_cab(b5, b4, epsilon=1e-8):
    """Chlorophyll a+b concentration approximation"""
    b5 = b5.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b5 + b4)
    return 100 * (b5 - b4) / (denominator + epsilon)

def extract_car(b2, b3, b4, epsilon=1e-8):
    """Carotenoid Reflectance Index"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    # Avoid division by zero/near-zero for reciprocals
    recip_b2 = 1 / (b2 + epsilon)
    recip_b3 = 1 / (b3 + epsilon)
    return (recip_b2 - recip_b3) * b4

def extract_cri1(b2, b5, epsilon=1e-8):
    """Carotenoid Reflectance Index 1"""
    b2 = b2.astype(np.float32)
    b5 = b5.astype(np.float32)
    # Avoid division by zero/near-zero for reciprocals
    recip_b2 = 1 / (b2 + epsilon)
    recip_b5 = 1 / (b5 + epsilon)
    return recip_b2 - recip_b5

def extract_cri2(b2, b7, epsilon=1e-8):
    """Carotenoid Reflectance Index 2"""
    b2 = b2.astype(np.float32)
    b7 = b7.astype(np.float32)
    # Avoid division by zero/near-zero for reciprocals
    recip_b2 = 1 / (b2 + epsilon)
    recip_b7 = 1 / (b7 + epsilon)
    return recip_b2 - recip_b7

def extract_pri(b2, b3, epsilon=1e-8):
    """Photochemical Reflectance Index"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    denominator = (b2 + b3)
    return (b2 - b3) / (denominator + epsilon)

def extract_lwi(b8, b11, epsilon=1e-8):
    """Leaf Water Index"""
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    denominator = (b8 + b11)
    return (b8 - b11) / (denominator + epsilon)

def extract_msi(b11, b8, epsilon=1e-8):
    """Moisture Stress Index"""
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    return b11 / (b8 + epsilon)

def extract_wi(b8, b3, epsilon=1e-8):
    """Water Index"""
    b8 = b8.astype(np.float32)
    b3 = b3.astype(np.float32)
    return b8 / (b3 + epsilon)

def extract_fe2(b4, b8, epsilon=1e-8):
    """Iron Oxide Ratio"""
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    return b4 / (b8 + epsilon)

def extract_fe3(b2, b4, epsilon=1e-8):
    """Iron Oxide Ratio 2"""
    b2 = b2.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b2 / (b4 + epsilon)

def extract_clay(b11, b12, epsilon=1e-8):
    """Clay Minerals Ratio"""
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return b11 / (b12 + epsilon)

# NEW ADDITIONAL INDICES

# Advanced Vegetation Indices
def extract_atsavi(b8, b4, L=0.08, epsilon=1e-8):
    """Adjusted Transformed Soil-Adjusted Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    numerator = 0.15 * (b8 - 0.15 * b4 - 0.08)
    denominator = (b8 + 0.15 * b4 + 0.08)
    return numerator / (denominator + epsilon)

def extract_gemi(b8, b4, epsilon=1e-8):
    """Global Environment Monitoring Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator_eta = (b8 + b4 + 0.5)
    # Avoid division by zero/near-zero for eta denominator
    eta = (2 * (b8**2 - b4**2) + 1.5 * b8 + 0.5 * b4) / (denominator_eta + epsilon)
    denominator_term2 = (1 - b4)
    safe_denominator_term2 = np.where(np.abs(denominator_term2) < epsilon, epsilon * np.sign(denominator_term2 + epsilon), denominator_term2)
    term2 = (b4 - 0.125) / safe_denominator_term2
    return eta * (1 - 0.25 * eta) - term2

def extract_ipvi(b8, b4, epsilon=1e-8):
    """Infrared Percentage Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b8 + b4)
    return b8 / (denominator + epsilon)

def extract_dvi(b8, b4):
    """Difference Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b8 - b4

def extract_rvi(b8, b4, epsilon=1e-8):
    """Ratio Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b8 / (b4 + epsilon)

def extract_sr(b8, b4, epsilon=1e-8):
    """Simple Ratio"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    return b8 / (b4 + epsilon)

def extract_tsavi(b8, b4, L=0.08, s=0.15, epsilon=1e-8):
    """Transformed Soil Adjusted Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    # Using user's formula structure
    numerator_user = s * (b8 - s * b4 - L)
    denominator_user = (s * b8 + b4 - s * L)
    return numerator_user / (denominator_user + epsilon)

def extract_msavi(b8, b4, epsilon=1e-8):
    """Modified Soil Adjusted Vegetation Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    arg_sqrt = (2 * b8 + 1)**2 - 8 * (b8 - b4)
    # Ensure sqrt argument is non-negative
    safe_arg_sqrt = np.maximum(arg_sqrt, 0)
    msavi = (2 * b8 + 1 - np.sqrt(safe_arg_sqrt)) / 2
    # Set result to NaN where arg_sqrt was negative
    msavi = np.where(arg_sqrt < 0, np.nan, msavi)
    return msavi

def extract_msavi2(b8, b4):
    """Modified Soil Adjusted Vegetation Index 2"""
    # Note: User's formula is identical to MSAVI. Re-using MSAVI implementation.
    # If MSAVI2 formula is different, this function should be updated.
    return extract_msavi(b8, b4) # Calls the extract_msavi function

def extract_gari(b8, b3, b4, b2, gamma=1.0, epsilon=1e-8):
    """Green Atmospherically Resistant Index"""
    b8 = b8.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    term = b3 - (b2 - b4)
    denominator = (b8 + term)
    return (b8 - term) / (denominator + epsilon)

def extract_gli(b3, b4, b2, epsilon=1e-8):
    """Green Leaf Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    denominator = (2 * b3 + b4 + b2)
    return (2 * b3 - b4 - b2) / (denominator + epsilon)

def extract_nli(b8, b4, epsilon=1e-8):
    """Non-Linear Index"""
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b8**2 + b4)
    return (b8**2 - b4) / (denominator + epsilon)

# Water Indices
def extract_awei_ns(b3, b11, b8, b12): # Linear combination
    """Automated Water Extraction Index (No Shadow)"""
    b3 = b3.astype(np.float32)
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    b12 = b12.astype(np.float32)
    return 4 * (b3 - b11) - 0.25 * b8 + 2.75 * b12

def extract_awei_sh(b2, b3, b11, b8, b12): # Linear combination
    """Automated Water Extraction Index (with Shadow)"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    b12 = b12.astype(np.float32)
    return b2 + 2.5 * b3 - 1.5 * (b8 + b11) - 0.25 * b12

def extract_wi2015(b3, b8, b11, b12): # Linear combination
    """Water Index 2015"""
    b3 = b3.astype(np.float32)
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return 1.7204 + 171 * b3 + 3 * b8 - 70 * b11 - 45 * b12

def extract_tcw(b2, b3, b4, b8, b11, b12): # Linear combination
    """Tasseled Cap Wetness"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return 0.0315 * b2 + 0.2021 * b3 + 0.3102 * b4 + 0.1595 * b8 - 0.6806 * b11 - 0.6109 * b12

def extract_tcb(b2, b3, b4, b8, b11, b12): # Linear combination
    """Tasseled Cap Brightness"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return 0.3037 * b2 + 0.2793 * b3 + 0.4743 * b4 + 0.5585 * b8 + 0.5082 * b11 + 0.1863 * b12

def extract_tcg(b2, b3, b4, b8, b11, b12): # Linear combination
    """Tasseled Cap Greenness"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return -0.2848 * b2 - 0.2435 * b3 - 0.5436 * b4 + 0.7243 * b8 + 0.0840 * b11 - 0.1800 * b12

# Geological and Soil Indices
def extract_ndti(b11, b12, epsilon=1e-8):
    """Normalized Difference Tillage Index"""
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    denominator = (b11 + b12)
    return (b11 - b12) / (denominator + epsilon)

def extract_sti(b11, b12, epsilon=1e-8):
    """Soil Tillage Index"""
    b11 = b11.astype(np.float32)
    b12 = b12.astype(np.float32)
    return b11 / (b12 + epsilon)

def extract_ci(b4, b3, epsilon=1e-8):
    """Coloration Index"""
    b4 = b4.astype(np.float32)
    b3 = b3.astype(np.float32)
    denominator = (b4 + b3)
    return (b4 - b3) / (denominator + epsilon)

def extract_hue(b2, b3, b4, epsilon=1e-8):
    """Hue Index"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (30.5 * (b3 - b2))
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    arg = 2 * (b4 - b3 - b2) / safe_denominator
    # Avoid log/atanh of non-finite values if arg is inf/nan
    arg = np.clip(arg, -1e10, 1e10) # Simple clipping to prevent inf, although arctan handles inf
    return np.arctan(arg)

def extract_sat(b2, b3, b4, epsilon=1e-8):
    """Saturation Index"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    max_band = np.maximum(np.maximum(b2, b3), b4)
    min_band = np.minimum(np.minimum(b2, b3), b4)
    safe_denominator = max_band + epsilon # Avoid division by zero max_band
    return (max_band - min_band) / safe_denominator

def extract_int(b2, b3, b4):
    """Intensity Index"""
    b2 = b2.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    return (b2 + b3 + b4) / 3.0

# Urban and Built-up Indices
def extract_ndbi(b11, b8, epsilon=1e-8):
    """Normalized Difference Built-up Index"""
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    denominator = (b11 + b8)
    return (b11 - b8) / (denominator + epsilon)

def extract_ebbi(b11, b8, b4, epsilon=1e-8):
    """Enhanced Built-Up and Bareness Index"""
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    b4 = b4.astype(np.float32)
    numerator = (b11 - b8)
    denominator_sqrt_arg = b11 + b4
    # Ensure sqrt argument is non-negative
    safe_denominator_sqrt_arg = np.maximum(denominator_sqrt_arg, 0)
    denominator = 10 * np.sqrt(safe_denominator_sqrt_arg)
    # Avoid division by zero/near-zero
    safe_denominator = np.where(np.abs(denominator) < epsilon, epsilon * np.sign(denominator + epsilon), denominator)
    ebbi = np.zeros_like(b11) # Default to zero
    valid_mask = safe_denominator != 0
    ebbi[valid_mask] = numerator[valid_mask] / safe_denominator[valid_mask]
    # Set result to NaN where arg was negative initially
    ebbi = np.where(denominator_sqrt_arg < 0, np.nan, ebbi)
    return ebbi


def extract_ui(b11, b8, epsilon=1e-8):
    """Urban Index"""
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    denominator = (b11 + b8)
    return (b11 - b8) / (denominator + epsilon)

def extract_ibi(b11, b8, b3, b4, epsilon=1e-8):
    """Index-based Built-up Index"""
    b11 = b11.astype(np.float32)
    b8 = b8.astype(np.float32)
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    ndbi_denominator = (b11 + b8)
    ndbi = (b11 - b8) / (ndbi_denominator + epsilon)

    savi_comp_denominator = (b8 + b4 + 0.5)
    savi_comp = ((b8 - b4) * 1.5) / (savi_comp_denominator + epsilon)

    mndwi_denominator = (b3 + b11)
    mndwi = (b3 - b11) / (mndwi_denominator + epsilon)

    avg_comp = (savi_comp + mndwi) / 2.0 # This might still be NaN if savi_comp or mndwi is NaN
    avg_comp = np.nan_to_num(avg_comp, nan=0.0) # Replace NaN with 0 for the average step safety

    numerator = (ndbi - avg_comp)
    denominator = (ndbi + avg_comp)
    # Avoid division by zero/near-zero for final IBI denominator
    return numerator / (denominator + epsilon)

# Snow and Ice Indices
def extract_s3(b3, b4, b8, epsilon=1e-8):
    """S3 Snow Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b8 = b8.astype(np.float32)
    numerator = b8 * (b4 - b3)
    denominator = (b8 + b4) * (b8 + b3)
    # Avoid division by zero/near-zero
    return numerator / (denominator + epsilon)

def extract_ndsii(b3, b4, epsilon=1e-8):
    """Normalized Difference Snow and Ice Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    denominator = (b3 + b4)
    return (b3 - b4) / (denominator + epsilon)

def extract_snowmap(b2, b11, epsilon=1e-8):
    """Snow Mapping Index"""
    b2 = b2.astype(np.float32)
    b11 = b11.astype(np.float32)
    denominator = (b2 + b11)
    return (b2 - b11) / (denominator + epsilon)

# Atmospheric Indices
def extract_vari(b3, b4, b2, epsilon=1e-8):
    """Visible Atmospherically Resistant Index"""
    b3 = b3.astype(np.float32)
    b4 = b4.astype(np.float32)
    b2 = b2.astype(np.float32)
    denominator = (b3 + b4 - b2)
    return (b3 - b4) / (denominator + epsilon)

def extract_gvmi(b8, b11, epsilon=1e-8):
    """Global Vegetation Moisture Index"""
    b8 = b8.astype(np.float32)
    b11 = b11.astype(np.float32)
    numerator = ((b8 + 0.1) - (b11 + 0.02))
    denominator = ((b8 + 0.1) + (b11 + 0.02))
    return numerator / (denominator + epsilon)

# Texture Features
def extract_glcm_features(band, distances=[1], angles=[0, 45, 90, 135]):
    """Extract Gray-Level Co-occurrence Matrix features"""
    features = []
    glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
    expected_features_count = len(distances) * len(angles) * len(glcm_props)

    try:
        # Handle potential NaNs/Inf before normalization
        band_clean = np.nan_to_num(band.astype(np.float32), nan=np.nanmedian(band) if np.nanmedian(band) is not np.nan else 0.0)
        band_clean = np.clip(band_clean, 0, 65535) # Clip to a reasonable 16-bit range as uint8 is target

        min_val = np.nanmin(band_clean)
        max_val = np.nanmax(band_clean)

        # Normalize to uint8 [0, 255]
        if max_val - min_val < 1e-8:
             band_norm = np.zeros(band_clean.shape, dtype=np.uint8)
        else:
             # Scale to 0-255 and convert to uint8
             band_norm = ((band_clean - min_val) / (max_val - min_val) * 255.0).astype(np.uint8)

        levels = 256 # Using 256 levels after converting to uint8

        for distance in distances:
            for angle in angles:
                try:
                    # Check if image is large enough for the distance
                    if band_norm.shape[0] <= distance or band_norm.shape[1] <= distance:
                         features.extend([0.0] * len(glcm_props))
                         continue

                    # Check if all values are the same after normalization (avoids GLCM error)
                    if np.min(band_norm) == np.max(band_norm):
                         features.extend([0.0] * len(glcm_props))
                         continue

                    # GLCM computation might raise warnings or errors on edge cases or weird inputs
                    try:
                        glcm = graycomatrix(band_norm, [distance], [np.radians(angle)],
                                         levels=levels, symmetric=True, normed=True)

                        if glcm.size > 0:
                             # Ensure glcm has the expected shape before accessing elements
                             if glcm.shape == (levels, levels, 1, 1):
                                contrast = graycoprops(glcm, 'contrast')[0, 0]
                                dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
                                homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
                                energy = graycoprops(glcm, 'energy')[0, 0]
                                correlation = graycoprops(glcm, 'correlation')[0, 0]
                                # Ensure finite values, replace with 0 if nan/inf
                                features.extend([
                                    np.nan_to_num(contrast, nan=0.0, posinf=0.0, neginf=0.0),
                                    np.nan_to_num(dissimilarity, nan=0.0, posinf=0.0, neginf=0.0),
                                    np.nan_to_num(homogeneity, nan=0.0, posinf=0.0, neginf=0.0),
                                    np.nan_to_num(energy, nan=0.0, posinf=0.0, neginf=0.0),
                                    np.nan_to_num(correlation, nan=0.0, posinf=0.0, neginf=0.0)
                                ])
                             else:
                                # Unexpected GLCM shape
                                features.extend([0.0] * len(glcm_props))

                        else:
                             # GLCM was empty (can happen for tiny images or certain inputs)
                             features.extend([0.0] * len(glcm_props))

                    except Exception as glcm_e:
                         # Catch errors specifically from graycomatrix/graycoprops
                         features.extend([0.0] * len(glcm_props))
                         # print(f"Error in GLCM for distance {distance}, angle {angle}: {glcm_e}") # Avoid printing

                except Exception as angle_e:
                    # Error during processing for a specific angle/distance iteration (less likely now)
                    features.extend([0.0] * len(glcm_props))
                    # print(f"Unexpected error in GLCM angle loop for distance {distance}, angle {angle}: {angle_e}") # Avoid printing


    except Exception as e:
        # General error during GLCM extraction setup or iteration
        features = [0.0] * expected_features_count
        # print(f"General error extracting GLCM features: {e}") # Avoid printing in parallel job

    # Ensure the correct number of features is returned even if errors occurred mid-loop
    if len(features) != expected_features_count:
         features = features[:expected_features_count] + [0.0] * (expected_features_count - len(features))

    return features

def extract_gabor_features(band, frequencies=[0.1, 0.3, 0.5]):
    """Extract Gabor filter features"""
    features = []
    gabor_stats = ['mean', 'std', 'max', 'min']
    expected_features_count = len(frequencies) * len(gabor_stats)

    try:
        band_float = band.astype(np.float32)
        band_clean = np.nan_to_num(band_float, nan=np.nanmean(band_float) if np.nanmean(band_float) is not np.nan else 0.0)

        for freq in frequencies:
            try:
                min_size_heuristic = 33 # Heuristic minimum size for Gabor kernel

                if band_clean.shape[0] < min_size_heuristic or band_clean.shape[1] < min_size_heuristic:
                     features.extend([0.0] * len(gabor_stats))
                     continue

                # Compute magnitude response
                # Gabor computation might raise errors for small sizes or specific inputs
                try:
                     real, imag = gabor(band_clean, frequency=freq)
                     magnitude = np.sqrt(real**2 + imag**2) # Magnitude response

                     # Calculate stats on the magnitude image
                     mean_val = np.nanmean(magnitude)
                     std_val = np.nanstd(magnitude)
                     max_val = np.nanmax(magnitude)
                     min_val = np.nanmin(magnitude)

                     # Ensure finite values, replace with 0 if nan/inf
                     features.extend([
                         np.nan_to_num(mean_val, nan=0.0, posinf=0.0, neginf=0.0),
                         np.nan_to_num(std_val, nan=0.0, posinf=0.0, neginf=0.0),
                         np.nan_to_num(max_val, nan=0.0, posinf=0.0, neginf=0.0),
                         np.nan_to_num(min_val, nan=0.0, posinf=0.0, neginf=0.0)
                     ])
                except Exception as gabor_e:
                     # Catch errors specifically from gabor or stats calculation
                     features.extend([0.0] * len(gabor_stats))
                     # print(f"Error in Gabor for freq {freq}: {gabor_e}") # Avoid printing

            except Exception as freq_e:
                # Error processing a specific frequency iteration (less likely now)
                features.extend([0.0] * len(gabor_stats))
                # print(f"Unexpected error in Gabor frequency loop for freq {freq}: {freq_e}") # Avoid printing


    except Exception as e:
        # General error during Gabor extraction setup or iteration
        features = [0.0] * expected_features_count
        # print(f"General error extracting Gabor features: {e}") # Avoid printing in parallel job

    if len(features) != expected_features_count:
         features = features[:expected_features_count] + [0.0] * (expected_features_count - len(features))

    # Ensure all features are finite (redundant with nan_to_num inside, but harmless)
    return [np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0) for f in features]

def extract_local_binary_pattern(band, P=8, R=1):
    """Extract Local Binary Pattern features (histogram of uniform patterns)"""
    features = []
    n_bins = P + 2 # Number of bins for uniform LBP (P neighbors) + 2 (0 and P+1)
    expected_features_count = n_bins

    try:
        band_float = band.astype(np.float32)
        band_clean = np.nan_to_num(band_float, nan=np.nanmean(band_float) if np.nanmean(band_float) is not np.nan else 0.0)

        min_size = int(2*R + 1)
        if band_clean.shape[0] < min_size or band_clean.shape[1] < min_size:
             return [0.0] * expected_features_count

        min_val = np.nanmin(band_clean)
        max_val = np.nanmax(band_clean)
        if max_val - min_val < 1e-8:
             band_norm_uint8 = np.zeros(band_clean.shape, dtype=np.uint8)
        else:
             band_norm_uint8 = ((band_clean - min_val) / (max_val - min_val) * 255.0).astype(np.uint8)

        # Compute LBP
        # LBP computation might raise errors on edge cases
        try:
            lbp_image = local_binary_pattern(band_norm_uint8, P=P, R=R, method='uniform')

            # Compute histogram of LBP codes
            # Histogram should be safe after LBP computation
            hist, _ = np.histogram(lbp_image, bins=n_bins, range=(0, n_bins), density=True) # density=True gives probabilities/normalized counts

            features.extend(hist.tolist())
        except Exception as lbp_e:
            # Catch errors specifically from local_binary_pattern or histogram
            features = [0.0] * expected_features_count
            # print(f"Error in LBP R={R}: {lbp_e}") # Avoid printing


    except Exception as e:
        # General error during LBP extraction setup
        features = [0.0] * expected_features_count
        # print(f"General error extracting LBP features: {e}") # Avoid printing in parallel job


    if len(features) != expected_features_count:
         features = features[:expected_features_count] + [0.0] * (expected_features_count - len(features))

    # Ensure features are finite
    return [np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0) for f in features]

def extract_morphological_features(band):
    """Extract morphological features based on binary thresholding"""
    features = []
    morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
    expected_features_count = len(morph_props)

    try:
        band_float = band.astype(np.float32)
        band_clean = np.nan_to_num(band_float, nan=np.nanmedian(band_float) if np.nanmedian(band_float) is not np.nan else 0.0)

        mean_val = np.nanmean(band_clean)
        if np.isnan(mean_val):
             return [0.0] * expected_features_count

        thresh = band_clean > mean_val # Boolean array

        structure = np.ones((3, 3), dtype=bool)

        if band_clean.shape[0] < 3 or band_clean.shape[1] < 3:
             return [0.0] * expected_features_count

        # Morphological operations might raise errors on edge cases
        try:
            opened = binary_opening(thresh, structure=structure)
            closed = binary_closing(thresh, structure=structure)
            eroded = binary_erosion(thresh, structure=structure)
            dilated = binary_dilation(thresh, structure=structure)

            total_thresh_area = np.sum(thresh)

            if total_thresh_area > 0:
                 features = [
                    np.sum(opened) / total_thresh_area,
                    np.sum(closed) / total_thresh_area,
                    np.sum(eroded) / total_thresh_area,
                    np.sum(dilated) / total_thresh_area
                ]
            else:
                 features = [0.0] * expected_features_count
        except Exception as morph_e:
            # Catch errors from binary morphology
            features = [0.0] * expected_features_count
            # print(f"Error in morphology: {morph_e}") # Avoid printing


    except Exception as e:
        # General error during morphological feature extraction setup
        features = [0.0] * expected_features_count
        # print(f"General error extracting morphological features: {e}") # Avoid printing in parallel job


    if len(features) != expected_features_count:
         features = features[:expected_features_count] + [0.0] * (expected_features_count - len(features))

    # Ensure features are finite
    return [np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0) for f in features]


def extract_entropy_features(band):
    """Extract entropy-based features (Shannon and local entropy stats)"""
    features = []
    entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
    expected_features_count = len(entropy_stats)

    try:
        band_float = band.astype(np.float32)
        band_clean = np.nan_to_num(band_float, nan=np.nanmedian(band_float) if np.nanmedian(band_float) is not np.nan else 0.0)
        band_clean = np.clip(band_clean, 0, 65535) # Clip to a reasonable 16-bit range as uint8 is target


        min_val = np.nanmin(band_clean)
        max_val = np.nanmax(band_clean)
        if max_val - min_val < 1e-8:
             band_norm_uint8 = np.zeros(band_clean.shape, dtype=np.uint8)
        else:
             band_norm_uint8 = ((band_clean - min_val) / (max_val - min_val) * 255.0).astype(np.uint8)


        # Calculate global Shannon entropy
        unique_vals = np.unique(band_norm_uint8)
        shannon_ent = 0.0 # Default
        if len(unique_vals) > 1: # Entropy is 0 if only one value is present
             # Skimage shannon_entropy works on array directly
             try:
                 shannon_ent = shannon_entropy(band_norm_uint8)
             except Exception as se_e:
                 shannon_ent = 0.0 # Handle errors


        # Calculate local entropy using skimage.filters.rank.entropy
        radius = 3 # Radius of the local neighborhood
        selem = disk(radius) # Structuring element for local entropy

        local_ent_mean = 0.0
        local_ent_std = 0.0

        if band_norm_uint8.shape[0] >= selem.shape[0] and band_norm_uint8.shape[1] >= selem.shape[1]:
            try:
                # skimage.filters.rank.entropy requires uint8 or uint16 input
                local_ent = entropy(band_norm_uint8, selem=selem)
                # Calculate stats (mean, std) on the local entropy map
                local_ent_mean = np.nanmean(local_ent)
                local_ent_std = np.nanstd(local_ent)
            except Exception as lre_e:
                 # Handle errors from rank entropy or stats
                 local_ent_mean = 0.0
                 local_ent_std = 0.0
                 # print(f"Error in local rank entropy radius {radius}: {lre_e}") # Avoid printing


        features = [shannon_ent, local_ent_mean, local_ent_std]

    except Exception as e:
        # General error during entropy extraction setup
        features = [0.0] * expected_features_count
        # print(f"General error extracting entropy features: {e}") # Avoid printing in parallel job


    if len(features) != expected_features_count:
         features = features[:expected_features_count] + [0.0] * (expected_features_count - len(features))

    # Ensure features are finite
    return [np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0) for f in features]


def extract_edge_features(band):
    """Extract edge detection features (sum, mean, std of edge maps)"""
    features = []
    edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
    edge_stats = ['sum', 'mean', 'std']
    expected_features_count = len(edge_types) * len(edge_stats)

    try:
        band_float = band.astype(np.float32)

        min_size = 3 # Minimum size for 3x3 kernel
        if band_float.shape[0] < min_size or band_float.shape[1] < min_size:
             return [0.0] * expected_features_count

        band_clean = np.nan_to_num(band_float, nan=np.nanmedian(band_float) if np.nanmedian(band_float) is not np.nan else 0.0)
        # Optional pre-smoothing can be added here if needed, but might affect performance

        # Apply edge detectors and calculate stats, handling potential errors for each
        edge_maps = {}
        try:
            edge_maps['canny'] = canny(band_clean).astype(np.float32)
        except Exception:
            edge_maps['canny'] = np.zeros_like(band_clean, dtype=np.float32)

        try:
            edge_maps['sobel'] = sobel(band_clean)
        except Exception:
            edge_maps['sobel'] = np.zeros_like(band_clean, dtype=np.float32)

        try:
            edge_maps['prewitt'] = prewitt(band_clean)
        except Exception:
            edge_maps['prewitt'] = np.zeros_like(band_clean, dtype=np.float32)

        try:
            edge_maps['roberts'] = roberts(band_clean)
        except Exception:
            edge_maps['roberts'] = np.zeros_like(band_clean, dtype=np.float32)

        # Calculate stats for each edge map
        def get_stats(edge_map):
            s = np.nansum(edge_map)
            m = np.nanmean(edge_map)
            sd = np.nanstd(edge_map)
            # Ensure finite values
            return [np.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0) for x in [s, m, sd]]

        # Append stats in the defined order
        for edge_type in edge_types:
             features.extend(get_stats(edge_maps.get(edge_type, np.zeros_like(band_clean, dtype=np.float32))))


    except Exception as e:
        # General error during edge feature extraction setup
        features = [0.0] * expected_features_count
        # print(f"General error extracting edge features: {e}") # Avoid printing in parallel job


    if len(features) != expected_features_count:
         features = features[:expected_features_count] + [0.0] * (expected_features_count - len(features))

    # Ensure features are finite
    return [np.nan_to_num(f, nan=0.0, posinf=0.0, neginf=0.0) for f in features]


# Helper function for batch stats
def batch_stats(arr_batch):
    """Computes mean, median, std, min, max for a batch of 2D arrays (N, H, W)."""
    # Ensure array is float and replace inf/nan with nan for stat calculation
    arr_batch = arr_batch.astype(np.float32)
    arr_clean = np.where(np.isfinite(arr_batch), arr_batch, np.nan)

    # Compute stats along the spatial dimensions (H, W) for each tile (N)
    # Use nan functions to handle any potential NaNs introduced or still present
    mean_vals = np.nanmean(arr_clean, axis=(1, 2))
    median_vals = np.nanmedian(arr_clean, axis=(1, 2))
    std_vals = np.nanstd(arr_clean, axis=(1, 2))
    min_vals = np.nanmin(arr_clean, axis=(1, 2))
    max_vals = np.nanmax(arr_clean, axis=(1, 2))

    # Replace any resulting NaNs/Infs in stats with 0
    mean_vals = np.nan_to_num(mean_vals, nan=0.0, posinf=0.0, neginf=0.0)
    median_vals = np.nan_to_num(median_vals, nan=0.0, posinf=0.0, neginf=0.0)
    std_vals = np.nan_to_num(std_vals, nan=0.0, posinf=0.0, neginf=0.0)
    min_vals = np.nan_to_num(min_vals, nan=0.0, posinf=0.0, neginf=0.0)
    max_vals = np.nan_to_num(max_vals, nan=0.0, posinf=0.0, neginf=0.0)


    # Stack stats into a single array (N, 5)
    return np.stack([mean_vals, median_vals, std_vals, min_vals, max_vals], axis=1)

################

def process_and_return_tiles(df, is_train=True, tile_size=TILE_SIZE):
    """
    Processes raster tiles from a DataFrame, pads them if necessary,
    and returns them as a stacked numpy array.
    Optionally returns labels for training data.
    Adds a tqdm progress bar for tile reading.
    """
    all_tiles = []
    all_labels = []
    skipped_count = 0

    # Add tqdm to the loop iterating through DataFrame rows
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Tiles", unit="file"):
        tile_path = row.get('tifPath') # Use .get for safety

        if is_train:
            # Ensure 'Target' column exists for training data
            if 'Target' not in row:
                 skipped_count += 1
                 continue
            label_vector = [row['Target']] # Assuming Target is a single value

        # Check if file exists and is valid
        if not tile_path or not os.path.exists(tile_path):
            skipped_count += 1
            continue

        try:
            with rasterio.open(tile_path) as src:
                # Read all bands as float32
                if src.count < 12:
                     skipped_count += 1
                     continue

                raster = src.read(out_dtype=np.float32) # (bands, height, width)

            raster[raster < 0] = np.nan # Simple example: set negative values to NaN

            # Pad or Crop if dimensions don't match tile_size
            bands, height, width = raster.shape

            if height != tile_size or width != tile_size:
                 raster_processed = np.full((bands, tile_size, tile_size), np.nan, dtype=np.float32)
                 copy_height = min(height, tile_size)
                 copy_width = min(width, tile_size)
                 raster_processed[:, :copy_height, :copy_width] = raster[:, :copy_height, :copy_width]
            else:
                raster_processed = raster # No padding needed

            all_tiles.append(raster_processed)

            if is_train:
                all_labels.append(label_vector)

        except rasterio.errors.RasterioIOError as e:
            skipped_count += 1
            continue
        except Exception as e:
            skipped_count += 1
            continue

    # Stack tiles and labels into numpy arrays
    if not all_tiles:
        print("Processed 0 valid tiles.")
        expected_bands = 12 # Default assumption
        if skipped_count > 0:
             print(f"Skipped {skipped_count} tiles.")
        return np.empty((0, expected_bands, tile_size, tile_size), dtype=np.float32), np.array([]) if is_train else None


    all_tiles_array = np.stack(all_tiles)  # shape: (N, C, H, W)
    all_labels_array = np.array(all_labels) if is_train else None  # shape: (N, labels)

    print(f"Successfully processed {len(all_tiles)} tiles.")
    if skipped_count > 0:
         print(f"Skipped {skipped_count} tiles due to errors or missing data.")
    return all_tiles_array, all_labels_array


# Define a function to extract texture features for a single tile
# This function will be called in parallel by joblib
def _extract_texture_features_for_one_tile(tile_index, all_data_map, texture_targets):
    """
    Helper function to extract texture features for a single tile across specified target layers.
    Designed to be called in parallel. Suppresses specific RuntimeWarnings locally.
    """
    # --- Set NumPy error handling to suppress warnings for invalid operations ---
    # This is necessary because joblib uses separate processes, each needing its own setting.
    # 'invalid' includes warnings like comparison with NaN, sqrt of negative, etc.
    # 'divide' includes warnings like division by zero.
    old_np_settings = np.seterr(invalid='ignore', divide='ignore')

    try:
        tile_texture_features = []
        glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        gabor_stats = ['mean', 'std', 'max', 'min']
        lbp_bins = 8 + 2 # P=8 uniform
        morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
        entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
        edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
        edge_stats = ['sum', 'mean', 'std']
        # Recalculate expected count for safety in padding/checking
        expected_features_count_per_tile = 0
        for target_name in texture_targets:
           glcm_feats_count = len([1])*len([0, 45, 90, 135])*len(glcm_props)
           gabor_feats_count = len([0.1, 0.3, 0.5])*len(gabor_stats)
           lbp_feats_count = lbp_bins
           morph_feats_count = len(morph_props)
           entropy_feats_count = len(entropy_stats)
           edge_feats_count = len(edge_types)*len(edge_stats)
           expected_features_count_per_tile += (glcm_feats_count + gabor_feats_count + lbp_feats_count + morph_feats_count + entropy_feats_count + edge_feats_count)


        for target_name in texture_targets:
            if target_name in all_data_map:
                target_data_2d = all_data_map[target_name][tile_index, :, :]
            else:
                # If a texture target isn't found (unexpected), append zero placeholders
                glcm_feats_count = len([1])*len([0, 45, 90, 135])*len(glcm_props)
                gabor_feats_count = len([0.1, 0.3, 0.5])*len(gabor_stats)
                lbp_feats_count = lbp_bins
                morph_feats_count = len(morph_props)
                entropy_feats_count = len(entropy_stats)
                edge_feats_count = len(edge_types)*len(edge_stats)
                tile_texture_features.extend([0.0] * (glcm_feats_count + gabor_feats_count + lbp_feats_count + morph_feats_count + entropy_feats_count + edge_feats_count))
                continue # Move to the next texture target


            # Apply all texture functions to this 2D data slice
            # Each function is responsible for handling potential errors/size issues on its own
            tile_texture_features.extend(extract_glcm_features(target_data_2d, distances=[1], angles=[0, 45, 90, 135]))
            tile_texture_features.extend(extract_gabor_features(target_data_2d, frequencies=[0.1, 0.3, 0.5]))
            tile_texture_features.extend(extract_local_binary_pattern(target_data_2d, P=8, R=1))
            tile_texture_features.extend(extract_morphological_features(target_data_2d))
            tile_texture_features.extend(extract_entropy_features(target_data_2d))
            tile_texture_features.extend(extract_edge_features(target_data_2d))

        # Final check within the worker function: ensure the number of features matches expected
        # This is a fallback for internal logic errors in texture functions not returning correct size consistently
        if len(tile_texture_features) != expected_features_count_per_tile:
             # Pad/trim if mismatch
             tile_texture_features = tile_texture_features[:expected_features_count_per_tile] + [0.0] * (expected_features_count_per_tile - len(tile_texture_features))


        return tile_texture_features # Return the feature list for this tile

    except Exception as e:
        # Catch any unexpected error during texture extraction for THIS tile
        # Return placeholder features if a severe error occurred for this tile
        # print(f"Severe error extracting texture features for tile index {tile_index}: {e}") # Avoid excessive printing
        # Recalculate expected count here for robustness in case of early failure
        glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        gabor_stats = ['mean', 'std', 'max', 'min']
        lbp_bins = 8 + 2 # P=8 uniform
        morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
        entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
        edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
        edge_stats = ['sum', 'mean', 'std']
        expected_features_count_per_tile = 0
        for target_name in texture_targets:
           glcm_feats_count = len([1])*len([0, 45, 90, 135])*len(glcm_props)
           gabor_feats_count = len([0.1, 0.3, 0.5])*len(gabor_stats)
           lbp_feats_count = lbp_bins
           morph_feats_count = len(morph_props)
           entropy_feats_count = len(entropy_stats)
           edge_feats_count = len(edge_types)*len(edge_stats)
           expected_features_count_per_tile += (glcm_feats_count + gabor_feats_count + lbp_feats_count + morph_feats_count + entropy_feats_count + edge_feats_count)

        return [0.0] * expected_features_count_per_tile # Return zeros if any error occurs for this tile


    finally:
        # --- Restore NumPy error handling settings for this process ---
        np.seterr(**old_np_settings)


def extract_features_from_tiles(tiles):
    """
    Extract statistical features and texture features using parallel processing.
    Texture extraction runs in parallel processes with warning suppression.
    Suppresses RuntimeWarnings during vectorized index calculations.
    """
    batch_size = 50
    num_tiles = tiles.shape[0]
    if num_tiles == 0:
        print("No tiles provided for feature extraction. Returning empty DataFrame.")
        # Build column list to return an empty DataFrame with correct columns
        # This needs to match the order used when tiles *are* present
        stat_names = ['mean', 'median', 'std', 'min', 'max']

        # Define band names (consistent order)
        band_names_in_order = [
             'B1_Coastal', 'B2_Blue', 'B3_Green', 'B4_Red', 'B5_RedEdge1', 'B6_RedEdge2',
             'B7_RedEdge3', 'B8_NIR', 'B8A_NarrowNIR', 'B9_WaterVapour', 'B11_SWIR1', 'B12_SWIR2'
        ]
        # Define index names in the order they are computed by add_index
        index_names_template = [
             'NDVI', 'SAVI', 'EVI', 'S2REP', 'CCCI', 'NDWI', # Original
             'NDRE', 'MMSR', 'GNDVI', 'EVI2', 'NGRDI', 'MNDWI', 'OSAVI', 'WDRVI', # Vegetation
             'TGI', 'GCVI', 'RGVI', 'ARVI', 'SIPI', 'RENDVI', 'MRESR', 'RYI',
             'NDYI', 'DYI', 'ACI', 'CVI', 'AVI', 'SI', 'BSI', 'MTCI',
             'NPCRI', 'MCARI', 'TCARI', 'PVI', 'BAI', 'MTVI2', 'NDSI', 'MRENDVI',
             'NDVI_RE', 'CI_RE', 'NDMI', 'TNDVI', 'VDVI', 'NBR', 'TVI', 'EXG',
             'PSRI', 'RDVI',
             'RATIO_B1_B3', 'RATIO_B1_B5', 'RATIO_B11_B12', # Ratios
             'RATIO_B5_B4', 'RATIO_B7_B5', 'RATIO_B7_B6', 'RATIO_B8_B4',
             'LAI', 'CAB', 'CAR', 'CRI1', 'CRI2', 'PRI', 'LWI', 'MSI', 'WI', # Biophysical/Other
             'FE2', 'FE3', 'CLAY', # Geological
             'ATSAVI', 'GEMI', 'IPVI', 'DVI', 'RVI', 'SR', 'TSAVI', 'MSAVI', 'MSAVI2', # Advanced Veg
             'GARI', 'GLI', 'NLI',
             'AWEI_NS', 'AWEI_SH', 'WI2015', 'TCW', 'TCB', 'TCG', # Water
             'NDTI', 'STI', 'CI_Coloration', 'HUE', 'SAT', 'INT', # Geological/Soil
             'NDBI', 'EBBI', 'UI', 'IBI', # Urban
             'S3', 'NDSII', 'SNOWMAP', # Snow
             'VARI', 'GVMI' # Atmospheric
        ]

        # Combine band and index names for stats columns
        all_data_names_template = band_names_in_order + index_names_template
        index_stat_columns = [f"{name}_{stat}" for name in all_data_names_template for stat in stat_names]

        # Texture feature column names (must match the order and types extracted in _extract_texture_features_for_one_tile)
        texture_columns = []
        texture_targets = ['B4_Red', 'B8_NIR', 'B11_SWIR1', 'NDVI'] # Must match names used in helper function
        glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        gabor_stats = ['mean', 'std', 'max', 'min']
        lbp_bins = 8 + 2 # P=8 uniform
        morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
        entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
        edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
        edge_stats = ['sum', 'mean', 'std']

        for target_name in texture_targets:
            for d in [1]: # GLCM distances
                for a in [0, 45, 90, 135]: # GLCM angles
                    texture_columns.extend([f"{target_name}_GLCM_d{d}a{a}_{prop}" for prop in glcm_props])
            for freq in [0.1, 0.3, 0.5]: # Gabor frequencies
                texture_columns.extend([f"{target_name}_Gabor_Freq{freq}_{stat}" for stat in gabor_stats])
            texture_columns.extend([f"{target_name}_LBP_Hist_bin{i}" for i in range(lbp_bins)])
            texture_columns.extend([f"{target_name}_Morph_{prop}" for prop in morph_props])
            texture_columns.extend([f"{target_name}_Entropy_{stat}" for stat in entropy_stats])
            for edge_type in edge_types:
                 texture_columns.extend([f"{target_name}_Edge_{edge_type}_{stat}" for stat in edge_stats])


        all_columns = index_stat_columns + texture_columns
        return pd.DataFrame([], columns=all_columns)


    print(f"Starting feature extraction for {num_tiles} tiles...")

    # --- Set NumPy error handling to suppress warnings for vectorized index calculations ---
    # This applies to the main process only. Workers handle their own settings.
    # Suppress 'invalid' (e.g., comparison with NaN, sqrt of negative) and 'divide' (e.g., division by zero).
    old_np_settings_main = np.seterr(invalid='ignore', divide='ignore')

    try:
        # --- Extract Bands (Batch) ---
        if tiles.shape[1] < 12:
            print(f"⚠️ Error: Tiles array has {tiles.shape[1]} bands, but at least 12 are required for feature extraction.")
            return pd.DataFrame([]) # Return empty DF on error

        # Extract bands by index - These operations are vectorized and usually fast
        b1 = tiles[:, 0, :, :]     # B1 (Coastal aerosol)
        b2 = tiles[:, 1, :, :]   # B2 (Blue)
        b3 = tiles[:, 2, :, :]  # B3 (Green)
        b4 = tiles[:, 3, :, :]    # B4 (Red)
        b5 = tiles[:, 4, :, :]     # B5 (Red Edge 1)
        b6 = tiles[:, 5, :, :]     # B6 (Red Edge 2)
        b7 = tiles[:, 6, :, :]     # B7 (Red Edge 3)
        b8 = tiles[:, 7, :, :]    # B8 (NIR)
        b8a = tiles[:, 8, :, :]    # B8A (Narrow NIR)
        b9 = tiles[:, 9, :, :]     # B9 (Water vapour)
        b11 = tiles[:, 10, :, :]   # B11 (SWIR 1)
        b12 = tiles[:, 11, :, :]   # B12 (SWIR 2)


        # --- Compute Indices (Batch) ---
        indices_batch = []
        index_names_in_order = []

        def add_index(func, name, *args):
            """Helper to compute index and add to list and names list, handling errors."""
            try:
                idx_result = func(*args)
                # Ensure result is float32 and handle potential NaNs/Infs from computation
                idx_result = np.asarray(idx_result, dtype=np.float32)
                indices_batch.append(idx_result)
                index_names_in_order.append(name)
            except Exception as e:
                # print(f"⚠️ Error computing index '{name}': {e}. Appending NaN array.") # Print less verbosely
                # Append an array of NaNs or a placeholder of the correct shape
                if tiles.shape[0] > 0:
                     placeholder = np.full_like(tiles[:, 0, :, :], np.nan, dtype=np.float32)
                else:
                     placeholder = np.empty((0, TILE_SIZE, TILE_SIZE), dtype=np.float32) # Fallback

                indices_batch.append(placeholder)
                index_names_in_order.append(name)


        print("Computing vegetation and other indices...")
        # These calculations are vectorized across tiles (N, H, W)
        # Warnings for invalid/divide operations will be suppressed by np.seterr above
        add_index(extract_ndvi, 'NDVI', b8, b4)
        add_index(extract_savi, 'SAVI', b8, b4)
        add_index(extract_evi, 'EVI', b8, b4, b2)
        add_index(extract_s2rep, 'S2REP', b4, b5, b6, b7)
        add_index(extract_ccci, 'CCCI', b4, b5, b8)
        add_index(extract_ndwi, 'NDWI', b3, b8)

        add_index(extract_ndre, 'NDRE', b8, b6)
        add_index(extract_mmsr, 'MMSR', b8, b4)
        add_index(extract_gndvi, 'GNDVI', b8, b3)
        add_index(extract_evi2, 'EVI2', b8, b4)
        add_index(extract_ngrdi, 'NGRDI', b3, b4)
        add_index(extract_mndwi, 'MNDWI', b3, b11)
        add_index(extract_osavi, 'OSAVI', b8, b4)
        add_index(extract_wdrvi, 'WDRVI', b8, b4)
        add_index(extract_tgi, 'TGI', b3, b4, b2)
        add_index(extract_gcvi, 'GCVI', b8, b3)
        add_index(extract_rgvi, 'RGVI', b1, b3, b4, b5, b7)
        add_index(extract_arvi, 'ARVI', b8, b4, b2)
        add_index(extract_sipi, 'SIPI', b8, b2, b4)
        add_index(extract_rendvi, 'RENDVI', b6, b5)
        add_index(extract_mresr, 'MRESR', b6, b1, b5)
        add_index(extract_ryi, 'RYI', b3, b2)
        add_index(extract_ndyi, 'NDYI', b3, b2)
        add_index(extract_dyi, 'DYI', b3, b2)
        add_index(extract_aci, 'ACI', b8, b4, b3)
        add_index(extract_cvi, 'CVI', b8, b3, b4)
        add_index(extract_avi, 'AVI', b8, b4)
        add_index(extract_si, 'SI', b2, b3, b4)
        add_index(extract_bsi, 'BSI', b11, b4, b8, b2)
        add_index(extract_mtci, 'MTCI', b6, b5, b4)
        add_index(extract_npcri, 'NPCRI', b4, b2)
        add_index(extract_mcari, 'MCARI', b5, b4, b3)
        add_index(extract_tcari, 'TCARI', b5, b4, b3)
        add_index(extract_pvi, 'PVI', b8, b4)
        add_index(extract_bai, 'BAI', b4, b8)
        add_index(extract_mtvi2, 'MTVI2', b8, b3, b4)
        add_index(extract_ndsi, 'NDSI', b3, b11)
        add_index(extract_mrendvi, 'MRENDVI', b6, b5, b1)
        add_index(extract_ndvi_re, 'NDVI_RE', b8, b5)
        add_index(extract_ci_re, 'CI_RE', b8, b5)
        add_index(extract_ndmi, 'NDMI', b8, b11)
        add_index(extract_tndvi, 'TNDVI', b8, b4)
        add_index(extract_vdvi, 'VDVI', b3, b4, b2)
        add_index(extract_nbr, 'NBR', b8, b11)
        add_index(extract_tvi, 'TVI', b6, b3, b4)
        add_index(extract_exg, 'EXG', b3, b4, b2)
        add_index(extract_psri, 'PSRI', b4, b2, b6)
        add_index(extract_rdvi, 'RDVI', b8, b4)
        add_index(extract_ratio_b1_b3, 'RATIO_B1_B3', b1, b3)
        add_index(extract_ratio_b1_b5, 'RATIO_B1_B5', b1, b5)
        add_index(extract_ratio_b11_b12, 'RATIO_B11_B12', b11, b12)
        add_index(extract_ratio_b5_b4, 'RATIO_B5_B4', b5, b4)
        add_index(extract_ratio_b7_b5, 'RATIO_B7_B5', b7, b5)
        add_index(extract_ratio_b7_b6, 'RATIO_B7_B6', b7, b6)
        add_index(extract_ratio_b8_b4, 'RATIO_B8_B4', b8, b4)
        add_index(extract_lai, 'LAI', b8, b4)
        add_index(extract_cab, 'CAB', b5, b4)
        add_index(extract_car, 'CAR', b2, b3, b4)
        add_index(extract_cri1, 'CRI1', b2, b5)
        add_index(extract_cri2, 'CRI2', b2, b7)
        add_index(extract_pri, 'PRI', b2, b3)
        add_index(extract_lwi, 'LWI', b8, b11)
        add_index(extract_msi, 'MSI', b11, b8)
        add_index(extract_wi, 'WI', b8, b3)
        add_index(extract_fe2, 'FE2', b4, b8)
        add_index(extract_fe3, 'FE3', b2, b4)
        add_index(extract_clay, 'CLAY', b11, b12)
        add_index(extract_atsavi, 'ATSAVI', b8, b4)
        add_index(extract_gemi, 'GEMI', b8, b4)
        add_index(extract_ipvi, 'IPVI', b8, b4)
        add_index(extract_dvi, 'DVI', b8, b4)
        add_index(extract_rvi, 'RVI', b8, b4)
        add_index(extract_sr, 'SR', b8, b4)
        add_index(extract_tsavi, 'TSAVI', b8, b4)
        add_index(extract_msavi, 'MSAVI', b8, b4)
        add_index(extract_msavi2, 'MSAVI2', b8, b4)
        add_index(extract_gari, 'GARI', b8, b3, b4, b2)
        add_index(extract_gli, 'GLI', b3, b4, b2)
        add_index(extract_nli, 'NLI', b8, b4)
        add_index(extract_awei_ns, 'AWEI_NS', b3, b11, b8, b12)
        add_index(extract_awei_sh, 'AWEI_SH', b2, b3, b11, b8, b12)
        add_index(extract_wi2015, 'WI2015', b3, b8, b11, b12)
        add_index(extract_tcw, 'TCW', b2, b3, b4, b8, b11, b12)
        add_index(extract_tcb, 'TCB', b2, b3, b4, b8, b11, b12)
        add_index(extract_tcg, 'TCG', b2, b3, b4, b8, b11, b12)
        add_index(extract_ndti, 'NDTI', b11, b12)
        add_index(extract_sti, 'STI', b11, b12)
        add_index(extract_ci, 'CI_Coloration', b4, b3)
        add_index(extract_hue, 'HUE', b2, b3, b4)
        add_index(extract_sat, 'SAT', b2, b3, b4)
        add_index(extract_int, 'INT', b2, b3, b4)
        add_index(extract_ndbi, 'NDBI', b11, b8)
        add_index(extract_ebbi, 'EBBI', b11, b8, b4)
        add_index(extract_ui, 'UI', b11, b8)
        add_index(extract_ibi, 'IBI', b11, b8, b3, b4)
        add_index(extract_s3, 'S3', b3, b4, b8)
        add_index(extract_ndsii, 'NDSII', b3, b4)
        add_index(extract_snowmap, 'SNOWMAP', b2, b11)
        add_index(extract_vari, 'VARI', b3, b4, b2)
        add_index(extract_gvmi, 'GVMI', b8, b11)


        # Prepend original bands to the list for batch stats
        bands_batch_data = [b1, b2, b3, b4, b5, b6, b7, b8, b8a, b9, b11, b12]
        band_names_in_order = [
            'B1_Coastal', 'B2_Blue', 'B3_Green', 'B4_Red', 'B5_RedEdge1', 'B6_RedEdge2',
            'B7_RedEdge3', 'B8_NIR', 'B8A_NarrowNIR', 'B9_WaterVapour', 'B11_SWIR1', 'B12_SWIR2'
        ]

        all_data_batch = bands_batch_data + indices_batch
        all_data_names = band_names_in_order + index_names_in_order # Combined list of names

        # Create a dictionary mapping the combined names to the batch arrays for texture lookup
        all_data_map = dict(zip(all_data_names, all_data_batch))


        # Compute stats for all bands and indices in a batch
        print("Calculating band and index statistics...")
        # This part is still vectorized NumPy and should be fast enough.
        all_stats_batch = [batch_stats(data) for data in all_data_batch]

        # Concatenate stats into a single array (N, total_bands_indices * 5)
        index_features_array = np.concatenate(all_stats_batch, axis=1)


        # --- Extract Texture Features (Per Tile) using Joblib ---
        texture_targets = ['B4_Red', 'B8_NIR', 'B11_SWIR1', 'NDVI'] # Layers for texture

        print(f"Extracting texture features per tile using {os.cpu_count()} cores...")

        # Use joblib.Parallel for the per-tile texture extraction
        # The _extract_texture_features_for_one_tile function handles its own errors and warning suppression
        texture_features_list = Parallel(n_jobs=-1, verbose=11)(
             delayed(_extract_texture_features_for_one_tile)(i, all_data_map, texture_targets)
             for i in range(num_tiles)
        )

        # Stack texture features and handle potential errors/padding
        if not texture_features_list:
             texture_features_array = np.empty((num_tiles, 0), dtype=np.float32)
        else:
             # Ensure consistent length before stacking
             # Recalculate expected texture feature count for one tile for robustness
             glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
             gabor_stats = ['mean', 'std', 'max', 'min']
             lbp_bins = 8 + 2
             morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
             entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
             edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
             edge_stats = ['sum', 'mean', 'std']
             expected_texture_features_count_per_tile = 0
             for target_name in texture_targets:
                glcm_feats_count = len([1])*len([0, 45, 90, 135])*len(glcm_props)
                gabor_feats_count = len([0.1, 0.3, 0.5])*len(gabor_stats)
                lbp_feats_count = lbp_bins
                morph_feats_count = len(morph_props)
                entropy_feats_count = len(entropy_stats)
                edge_feats_count = len(edge_types)*len(edge_stats)
                expected_texture_features_count_per_tile += (glcm_feats_count + gabor_feats_count + lbp_feats_count + morph_feats_count + entropy_feats_count + edge_feats_count)

             expected_len = expected_texture_features_count_per_tile # Use the pre-calculated length


             # Pad any feature lists that are shorter than the expected length
             # (Features that failed extraction in a worker process will be [0.0] * expected_len already due to _extract_texture_features_for_one_tile's error handling)
             # This step primarily ensures consistency if the expected_len calculation differs between main and worker, or if a rare edge case occurred.
             padded_features_list = []
             for features in texture_features_list:
                 if len(features) < expected_len:
                     padded_features_list.append(features + [0.0] * (expected_len - len(features)))
                 elif len(features) > expected_len:
                      padded_features_list.append(features[:expected_len])
                 else:
                     padded_features_list.append(features)

             texture_features_array = np.array(padded_features_list, dtype=np.float32)


        # --- Combine All Features ---
        all_features_array = np.concatenate([index_features_array, texture_features_array], axis=1)

        # --- Create Column Names ---
        stat_names = ['mean', 'median', 'std', 'min', 'max']
        # List of all bands and indices *names* for the stats part
        all_data_names_actual = band_names_in_order + index_names_in_order # Use the actual names list generated

        index_stat_columns = [f"{name}_{stat}" for name in all_data_names_actual for stat in stat_names]

        # List of texture feature names: Use the texture_targets list and feature function structures
        texture_columns = []
        texture_targets_actual = ['B4_Red', 'B8_NIR', 'B11_SWIR1', 'NDVI'] # Re-declare for clarity
        glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
        gabor_stats = ['mean', 'std', 'max', 'min']
        lbp_bins = 8 + 2
        morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
        entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
        edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
        edge_stats = ['sum', 'mean', 'std']

        for target_name in texture_targets_actual:
            for d in [1]:
                for a in [0, 45, 90, 135]:
                    texture_columns.extend([f"{target_name}_GLCM_d{d}a{a}_{prop}" for prop in glcm_props])
            for freq in [0.1, 0.3, 0.5]:
                texture_columns.extend([f"{target_name}_Gabor_Freq{freq}_{stat}" for stat in gabor_stats])
            texture_columns.extend([f"{target_name}_LBP_Hist_bin{i}" for i in range(lbp_bins)])
            texture_columns.extend([f"{target_name}_Morph_{prop}" for prop in morph_props])
            texture_columns.extend([f"{target_name}_Entropy_{stat}" for stat in entropy_stats])
            for edge_type in edge_types:
                 texture_columns.extend([f"{target_name}_Edge_{edge_type}_{stat}" for stat in edge_stats])


        all_columns = index_stat_columns + texture_columns

        # Final check on column count
        if all_features_array.shape[1] != len(all_columns):
             print(f"⚠️ CRITICAL MISMATCH: Extracted features count ({all_features_array.shape[1]}) does not match column names count ({len(all_columns)}).")
             print(f"  Expected Index Stats Columns Count: {len(all_data_names_actual) * len(stat_names)}")
             print(f"  Actual Index Features Array Columns: {index_features_array.shape[1]}")
             print(f"  Expected Texture Columns Count: {expected_texture_features_count_per_tile}")
             print(f"  Actual Texture Features Array Columns: {texture_features_array.shape[1]}")
             print(f"  Total Expected Columns: {len(all_columns)}")
             print(f"  Total Actual Columns: {all_features_array.shape[1]}")
             # Fallback to generic names if mismatch
             return pd.DataFrame(all_features_array, columns=[f'feature_{i}' for i in range(all_features_array.shape[1])])

        print(f"Extracted {all_features_array.shape[1]} features for {num_tiles} tiles.")
        return pd.DataFrame(all_features_array, columns=all_columns)

    finally:
        # --- Restore NumPy error handling settings for the main process ---
        np.seterr(**old_np_settings_main)

In [25]:
# Determine the actual local path to the S2Images folder within the downloaded dataset
# Based on the original path '/kaggle/input/ivorycoast-byte-sizedagric/Côte d’Ivoire Byte-Sized Agric/S2Images/S2Images/'
# The structure inside the downloaded 'paulnkamau_ivorycoast_byte_sizedagric' directory should mirror this.
# The base path for the S2Images folder is likely:
local_image_folder_base = os.path.join(
    paulnkamau_ivorycoast_byte_sizedagric_path,
    'Côte d’Ivoire Byte-Sized Agric', # Adjust this folder name if it differs in the downloaded path
    'S2Images',
    'S2Images'
)

# The prefix we want to remove from the original tifPath values is:
kaggle_image_path_prefix = '/kaggle/input/ivorycoast-byte-sizedagric/Côte d’Ivoire Byte-Sized Agric/S2Images/S2Images/'

# Replace the Kaggle prefix with the local base path for the image folder
# Add os.sep to the local path to ensure it has a trailing slash appropriate for the OS
train["tifPath"] = train["tifPath"].str.replace(
    kaggle_image_path_prefix,
    local_image_folder_base + os.sep,
    regex=False
)

test["tifPath"] = test["tifPath"].str.replace(
    kaggle_image_path_prefix,
    local_image_folder_base + os.sep,
    regex=False
)

print("tifPath columns updated to local paths.")
# Optional: Print a sample tifPath to verify
print("\nSample train tifPath after correction:")
print(train["tifPath"].iloc[0])
print("\nSample test tifPath after correction:")
print(test["tifPath"].iloc[0])


# --- Now continue with your existing code, ensuring you have the feature extraction functions defined ---

# Assuming TILE_SIZE is defined globally, e.g., from MAX_TILE_SIZE calculation
try:
    TILE_SIZE
except NameError:
    print("TILE_SIZE not defined. Using default 326.")
    TILE_SIZE = 326 # Default if MAX_TILE_SIZE was not computed earlier

def process_and_return_tiles_with_df(df, is_train=True, tile_size=TILE_SIZE):
    # ... (your existing process_and_return_tiles_with_df function code) ...
    """
    Processes raster tiles from a DataFrame, padding if necessary.
    Returns the subset of the original DataFrame corresponding to processed tiles,
    the stacked numpy array of tiles, and labels (if is_train).
    Handles missing paths and basic raster errors.
    Includes tqdm progress bar and garbage collection.
    """
    all_tiles = []
    all_labels = []
    processed_rows = [] # Store the original DataFrame rows that were successfully processed
    skipped_count = 0

    # Use tqdm on the provided DataFrame rows
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Tiles", unit="file"):
        tile_path = row.get('tifPath')

        label_vector = None # Default for test data
        if is_train:
            # Ensure 'Target' column exists for training data
            if 'Target' not in row:
                 skipped_count += 1
                 # print(f"Skipping row {index}: 'Target' column not found.") # Debugging
                 continue
            label_vector = row['Target'] # Store as single value (string: 'Cocoa', 'Palm', 'Rubber')


        # Check if file exists and is valid
        if not tile_path or not os.path.exists(str(tile_path)):
            skipped_count += 1
            # print(f"Skipping row {index}: Missing or invalid file path: {tile_path}") # Debugging
            continue

        try:
            with rasterio.open(str(tile_path)) as src:
                # Ensure the tile has at least 12 bands as expected by feature extraction
                if src.count < 12:
                     skipped_count += 1
                     # print(f"Skipping row {index} ({tile_path}): Expected at least 12 bands, found {src.count}.") # Debugging
                     continue

                # Read all bands as float32
                raster = src.read(out_dtype=np.float32) # (bands, height, width)

            # Handle potential NoData values, often represented as specific numbers like -9999
            # A simple way is to replace common NoData values or just keep NaNs
            raster[raster < 0] = np.nan # Set negative values to NaN


            # Pad or Crop if dimensions don't match tile_size
            bands, height, width = raster.shape

            if height != tile_size or width != tile_size:
                 raster_processed = np.full((bands, tile_size, tile_size), np.nan, dtype=np.float32) # Use NaN for padding
                 copy_height = min(height, tile_size)
                 copy_width = min(width, tile_size)
                 raster_processed[:, :copy_height, :copy_width] = raster[:, :copy_height, :copy_width]
            else:
                raster_processed = raster # No padding needed

            all_tiles.append(raster_processed)
            processed_rows.append(row) # Append the original row for later DataFrame reconstruction

            if is_train:
                all_labels.append(label_vector) # Append single label

        except rasterio.errors.RasterioIOError:
            # print(f"Rasterio error processing {tile_path}: {e}") # Suppress printing
            skipped_count += 1
            continue
        except Exception:
            # print(f"Unexpected error processing {tile_path}: {e}") # Suppress printing
            skipped_count += 1
            continue

    # Stack tiles and create processed DataFrame
    if not all_tiles:
        print("Processed 0 valid tiles.")
        # Return empty DataFrames with original columns, and empty tiles/labels arrays
        # Need a way to get columns if no rows were processed. Let's return empty stuff.
        empty_df = df.iloc[[]].copy()
        empty_tiles = np.empty((0, 12, tile_size, tile_size), dtype=np.float32) # Assume 12 bands if 0 processed
        empty_labels = np.array([]) if is_train else None
        if skipped_count > 0:
             print(f"Skipped {skipped_count} tiles.")
        return empty_df, empty_tiles, empty_labels


    processed_df = pd.DataFrame(processed_rows).reset_index(drop=True) # Create DF from collected rows
    all_tiles_array = np.stack(all_tiles)  # shape: (N, C, H, W)

    # Clean up memory from the list of individual tile arrays
    del all_tiles
    gc.collect()

    if is_train:
         all_labels_array = np.array(all_labels) # shape: (N,)
         del all_labels
         gc.collect()
    else:
         all_labels_array = None # Keep as None for test

    print(f"Successfully processed {len(processed_df)} tiles.")
    if skipped_count > 0:
         print(f"Skipped {skipped_count} tiles due to errors or missing data.")

    # Final consistency checks
    if is_train and len(processed_df) != len(all_labels_array):
        raise ValueError(f"Mismatch between processed DataFrame rows ({len(processed_df)}) and labels array size ({len(all_labels_array)}) for train.")
    if len(processed_df) != all_tiles_array.shape[0]:
        raise ValueError(f"Mismatch between processed DataFrame rows ({len(processed_df)}) and tiles array size ({all_tiles_array.shape[0]}).")


    return processed_df, all_tiles_array, all_labels_array # Return the processed DF slice too

# Assuming extract_features_from_tiles and its helpers are defined previously


# --- Filter DataFrames by Month ---
print("\nFiltering train and test data by month...")
train_dec_jan_feb = train[train['month'].isin(['Dec', 'Jan', 'Feb'])].copy()
test_jan_feb = test[test['month'].isin(['Jan', 'Feb'])].copy()

print(f"Train data for Dec, Jan, Feb: {len(train_dec_jan_feb)} rows")
print(f"Test data for Jan, Feb: {len(test_jan_feb)} rows")

tifPath columns updated to local paths.

Sample train tifPath after correction:
/root/.cache/kagglehub/datasets/paulnkamau/ivorycoast-byte-sizedagric/versions/1/Côte d’Ivoire Byte-Sized Agric/S2Images/S2Images/train/s2_Rubber_ID_h14T0B_2024_01.tif

Sample test tifPath after correction:
/root/.cache/kagglehub/datasets/paulnkamau/ivorycoast-byte-sizedagric/versions/1/Côte d’Ivoire Byte-Sized Agric/S2Images/S2Images/test/s2_Unknown_ID_731818_2024_01.tif

Filtering train and test data by month...
Train data for Dec, Jan, Feb: 2859 rows
Test data for Jan, Feb: 564 rows


In [ ]:
# Import tqdm for progress bars
from tqdm.auto import tqdm

# Import garbage collection
import gc

# Assuming TILE_SIZE is defined globally, e.g., from MAX_TILE_SIZE calculation
try:
    TILE_SIZE
except NameError:
    print("TILE_SIZE not defined. Using default 326.")
    TILE_SIZE = 326 # Default if MAX_TILE_SIZE was not computed earlier

def process_and_return_tiles_with_df(df, is_train=True, tile_size=TILE_SIZE):
    """
    Processes raster tiles from a DataFrame, padding if necessary.
    Returns the subset of the original DataFrame corresponding to processed tiles,
    the stacked numpy array of tiles, and labels (if is_train).
    Handles missing paths and basic raster errors.
    Includes tqdm progress bar and garbage collection.
    """
    all_tiles = []
    all_labels = []
    processed_rows = [] # Store the original DataFrame rows that were successfully processed
    skipped_count = 0

    # Use tqdm on the provided DataFrame rows
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Tiles", unit="file"):
        tile_path = row.get('tifPath')

        label_vector = None # Default for test data
        if is_train:
            # Ensure 'Target' column exists for training data
            if 'Target' not in row:
                 skipped_count += 1
                 # print(f"Skipping row {index}: 'Target' column not found.") # Debugging
                 continue
            label_vector = row['Target'] # Store as single value (string: 'Cocoa', 'Palm', 'Rubber')


        # Check if file exists and is valid
        if not tile_path or not os.path.exists(str(tile_path)):
            skipped_count += 1
            # print(f"Skipping row {index}: Missing or invalid file path: {tile_path}") # Debugging
            continue

        try:
            with rasterio.open(str(tile_path)) as src:
                # Ensure the tile has at least 12 bands as expected by feature extraction
                if src.count < 12:
                     skipped_count += 1
                     # print(f"Skipping row {index} ({tile_path}): Expected at least 12 bands, found {src.count}.") # Debugging
                     continue

                # Read all bands as float32
                raster = src.read(out_dtype=np.float32) # (bands, height, width)

            # Handle potential NoData values, often represented as specific numbers like -9999
            # A simple way is to replace common NoData values or just keep NaNs
            raster[raster < 0] = np.nan # Set negative values to NaN


            # Pad or Crop if dimensions don't match tile_size
            bands, height, width = raster.shape

            if height != tile_size or width != tile_size:
                 raster_processed = np.full((bands, tile_size, tile_size), np.nan, dtype=np.float32) # Use NaN for padding
                 copy_height = min(height, tile_size)
                 copy_width = min(width, tile_size)
                 raster_processed[:, :copy_height, :copy_width] = raster[:, :copy_height, :copy_width]
            else:
                raster_processed = raster # No padding needed

            all_tiles.append(raster_processed)
            processed_rows.append(row) # Append the original row for later DataFrame reconstruction

            if is_train:
                all_labels.append(label_vector) # Append single label

        except rasterio.errors.RasterioIOError:
            # print(f"Rasterio error processing {tile_path}: {e}") # Suppress printing
            skipped_count += 1
            continue
        except Exception:
            # print(f"Unexpected error processing {tile_path}: {e}") # Suppress printing
            skipped_count += 1
            continue

    # Stack tiles and create processed DataFrame
    if not all_tiles:
        print("Processed 0 valid tiles.")
        # Return empty DataFrames with original columns, and empty tiles/labels arrays
        # Need a way to get columns if no rows were processed. Let's return empty stuff.
        empty_df = df.iloc[[]].copy()
        empty_tiles = np.empty((0, 12, tile_size, tile_size), dtype=np.float32) # Assume 12 bands if 0 processed
        empty_labels = np.array([]) if is_train else None
        if skipped_count > 0:
             print(f"Skipped {skipped_count} tiles.")
        return empty_df, empty_tiles, empty_labels


    processed_df = pd.DataFrame(processed_rows).reset_index(drop=True) # Create DF from collected rows
    all_tiles_array = np.stack(all_tiles)  # shape: (N, C, H, W)

    # Clean up memory from the list of individual tile arrays
    del all_tiles
    gc.collect()

    if is_train:
         all_labels_array = np.array(all_labels) # shape: (N,)
         del all_labels
         gc.collect()
    else:
         all_labels_array = None # Keep as None for test

    print(f"Successfully processed {len(processed_df)} tiles.")
    if skipped_count > 0:
         print(f"Skipped {skipped_count} tiles due to errors or missing data.")

    # Final consistency checks
    if is_train and len(processed_df) != len(all_labels_array):
        raise ValueError(f"Mismatch between processed DataFrame rows ({len(processed_df)}) and labels array size ({len(all_labels_array)}) for train.")
    if len(processed_df) != all_tiles_array.shape[0]:
        raise ValueError(f"Mismatch between processed DataFrame rows ({len(processed_df)}) and tiles array size ({all_tiles_array.shape[0]}).")


    return processed_df, all_tiles_array, all_labels_array # Return the processed DF slice too


# --- Filter DataFrames by Month ---
print("Filtering train and test data by month...")
train_dec_jan_feb = train[train['month'].isin(['Dec', 'Jan', 'Feb'])].copy()
test_jan_feb = test[test['month'].isin(['Jan', 'Feb'])].copy()

print(f"Train data for Dec, Jan, Feb: {len(train_dec_jan_feb)} rows")
print(f"Test data for Jan, Feb: {len(test_jan_feb)} rows")


# --- Process Tiles and Extract Features ---
# Call the modified processing function
print("\nProcessing train data and collecting tiles...")
train_processed_df, train_tiles_array, train_labels = process_and_return_tiles_with_df(train_dec_jan_feb, is_train=True)

print("\nProcessing test data and collecting tiles...")
test_processed_df, test_tiles_array, test_labels = process_and_return_tiles_with_df(test_jan_feb, is_train=False) # test_labels should be None


print(f"\nShape of train_processed_df: {train_processed_df.shape}")
print(f"Shape of train_tiles_array: {train_tiles_array.shape}")
print(f"Shape of train_labels: {train_labels.shape if train_labels is not None else None}")

print(f"\nShape of test_processed_df: {test_processed_df.shape}")
print(f"Shape of test_tiles_array: {test_tiles_array.shape}")
print(f"Shape of test_labels: {test_labels}") # Should be None

# Extract Features from the collected tiles
# This function should be defined previously in your notebook
# batch_size for feature extraction is distinct from the batching in process_and_return_tiles if used there
feature_extract_batch_size = 50 # Adjust based on memory
batch_size = 50

print("\nExtracting features for train data...")
# Check if there are tiles to process
if train_tiles_array.shape[0] > 0:
    X_train_features = extract_features_from_tiles(train_tiles_array)
else:
    print("No train tiles to extract features from. Returning empty DataFrame.")
    # Need to get expected columns somehow - perhaps run extract_features_from_tiles on a dummy empty array?
    # Or manually define the expected columns. Manual definition is safer for consistency.
    # Manually defining column names here (must match extract_features_from_tiles output structure)
    stat_names = ['mean', 'median', 'std', 'min', 'max']
    band_names_template = [
         'B1_Coastal', 'B2_Blue', 'B3_Green', 'B4_Red', 'B5_RedEdge1', 'B6_RedEdge2',
         'B7_RedEdge3', 'B8_NIR', 'B8A_NarrowNIR', 'B9_WaterVapour', 'B11_SWIR1', 'B12_SWIR2'
    ]
    index_names_template = [
         'NDVI', 'SAVI', 'EVI', 'S2REP', 'CCCI', 'NDWI',
         'NDRE', 'MMSR', 'GNDVI', 'EVI2', 'NGRDI', 'MNDWI', 'OSAVI', 'WDRVI',
         'TGI', 'GCVI', 'RGVI', 'ARVI', 'SIPI', 'RENDVI', 'MRESR', 'RYI',
         'NDYI', 'DYI', 'ACI', 'CVI', 'AVI', 'SI', 'BSI', 'MTCI',
         'NPCRI', 'MCARI', 'TCARI', 'PVI', 'BAI', 'MTVI2', 'NDSI', 'MRENDVI',
         'NDVI_RE', 'CI_RE', 'NDMI', 'TNDVI', 'VDVI', 'NBR', 'TVI', 'EXG',
         'PSRI', 'RDVI',
         'RATIO_B1_B3', 'RATIO_B1_B5', 'RATIO_B11_B12',
         'RATIO_B5_B4', 'RATIO_B7_B5', 'RATIO_B7_B6', 'RATIO_B8_B4',
         'LAI', 'CAB', 'CAR', 'CRI1', 'CRI2', 'PRI', 'LWI', 'MSI', 'WI',
         'FE2', 'FE3', 'CLAY',
         'ATSAVI', 'GEMI', 'IPVI', 'DVI', 'RVI', 'SR', 'TSAVI', 'MSAVI', 'MSAVI2',
         'GARI', 'GLI', 'NLI',
         'AWEI_NS', 'AWEI_SH', 'WI2015', 'TCW', 'TCB', 'TCG',
         'NDTI', 'STI', 'CI_Coloration', 'HUE', 'SAT', 'INT',
         'NDBI', 'EBBI', 'UI', 'IBI',
         'S3', 'NDSII', 'SNOWMAP',
         'VARI', 'GVMI'
    ]
    all_data_names_template = band_names_template + index_names_template
    index_stat_columns = [f"{name}_{stat}" for name in all_data_names_template for stat in stat_names]

    texture_columns_template = [] # Recalculate texture columns template
    texture_targets = ['B4_Red', 'B8_NIR', 'B11_SWIR1', 'NDVI']
    glcm_props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']
    gabor_stats = ['mean', 'std', 'max', 'min']
    lbp_bins = 8 + 2
    morph_props = ['OpeningRatio', 'ClosingRatio', 'ErosionRatio', 'DilationRatio']
    entropy_stats = ['Shannon', 'LocalMean', 'LocalStd']
    edge_types = ['canny', 'sobel', 'prewitt', 'roberts']
    edge_stats = ['sum', 'mean', 'std']
    for target_name in texture_targets:
        for d in [1]:
            for a in [0, 45, 90, 135]:
                texture_columns_template.extend([f"{target_name}_GLCM_d{d}a{a}_{prop}" for prop in glcm_props])
        for freq in [0.1, 0.3, 0.5]:
            texture_columns_template.extend([f"{target_name}_Gabor_Freq{freq}_{stat}" for stat in gabor_stats])
        texture_columns_template.extend([f"{target_name}_LBP_Hist_bin{i}" for i in range(lbp_bins)])
        texture_columns_template.extend([f"{target_name}_Morph_{prop}" for prop in morph_props])
        texture_columns_template.extend([f"{target_name}_Entropy_{stat}" for stat in entropy_stats])
        for edge_type in edge_types:
             texture_columns_template.extend([f"{target_name}_Edge_{edge_type}_{stat}" for stat in edge_stats])
    all_columns_template = index_stat_columns + texture_columns_template

    X_train_features = pd.DataFrame([], columns=all_columns_template) # Empty DF with expected columns

# Release train tiles memory immediately after feature extraction
del train_tiles_array
gc.collect()


print("\nExtracting features for test data...")
# Check if there are tiles to process
if test_tiles_array.shape[0] > 0:
    X_test_features = extract_features_from_tiles(test_tiles_array)
else:
    print("No test tiles to extract features from. Returning empty DataFrame.")
    # Use the same column template as train
    X_test_features = pd.DataFrame([], columns=all_columns_template) # Empty DF with expected columns


# Release test tiles memory immediately after feature extraction
del test_tiles_array
gc.collect()

print(f"\nShape of X_train_features: {X_train_features.shape}")
print(f"Shape of X_test_features: {X_test_features.shape}")


# --- Handle NaNs/Infs in Features (Imputation) ---
print("\nHandling NaN/Inf in features using median imputation from train data...")

# Check if train features are empty - if so, imputation is not possible/needed
if X_train_features.shape[0] == 0:
    print("Train features are empty, skipping imputation.")
    # No imputation possible. X_test_features will also likely be empty or full of NaNs.
    # K-NN and prediction will be skipped later.

else:
    # Check for column mismatch between train and test features (shouldn't happen if templates match)
    if not X_train_features.columns.equals(X_test_features.columns):
        print("⚠️ Warning: Column mismatch between train and test features. Attempting to align.")
        # Align columns - this might drop columns unique to one set or fill with NaNs
        common_cols = X_train_features.columns.intersection(X_test_features.columns)
        X_train_features = X_train_features[common_cols].copy()
        X_test_features = X_test_features[common_cols].copy()
        print(f"Aligned to {len(common_cols)} common columns.")


    # Replace Inf with NaN first in both train and test features
    X_train_features = X_train_features.replace([np.inf, -np.inf], np.nan)
    X_test_features = X_test_features.replace([np.inf, -np.inf], np.nan)

    # Calculate medians *only* on the training data to avoid data leakage
    medians = X_train_features.median()

    # Identify columns where median is NaN (all train values were NaN)
    cols_with_all_nan_train = medians[medians.isna()].index.tolist()

    # Impute NaNs in both train and test features using train medians
    # fillna returns a new DataFrame, reassign
    X_train_features = X_train_features.fillna(medians)
    X_test_features = X_test_features.fillna(medians)

    # For any columns where median was NaN (all training values were NaN/Inf), fill remaining NaNs with 0
    if cols_with_all_nan_train:
        print(f"Imputing remaining NaNs (from all-NaN/Inf train columns) with 0 for {len(cols_with_all_nan_train)} columns.")
        X_train_features[cols_with_all_nan_train] = X_train_features[cols_with_all_nan_train].fillna(0)
        X_test_features[cols_with_all_nan_train] = X_test_features[cols_with_all_nan_train].fillna(0)

    # Final check that no NaNs/Infs remain
    if X_train_features.isnull().any().any() or (X_train_features == np.inf).any().any() or (X_train_features == -np.inf).any().any():
         print("CRITICAL ERROR: NaN/Inf found in X_train_features after cleaning.")
    if X_test_features.isnull().any().any() or (X_test_features == np.inf).any().any() or (X_test_features == -np.inf).any().any():
         print("CRITICAL ERROR: NaN/Inf found in X_test_features after cleaning.")

print(f"\nShape of X_train_features after cleaning: {X_train_features.shape}")
print(f"Shape of X_test_features after cleaning: {X_test_features.shape}")


# --- Filter Train Data based on Test Similarity (K-NN approach) ---
from sklearn.neighbors import NearestNeighbors

K_filter = 50 # Number of neighbors to find for filtering train data
print(f"\nFinding {K_filter}-nearest neighbors in train for each test image to filter train data...")

# Check if there's enough data to perform K-NN
if X_train_features.shape[0] == 0 or X_test_features.shape[0] == 0 or K_filter > X_train_features.shape[0]:
     print("Cannot perform K-NN filtering: insufficient data or K_filter is too large.")
     # If filtering is not possible, the "filtered train data" is either empty or maybe the original?
     # Based on the goal, an empty filtered set seems consistent if there's no test data to match.
     filtered_train_data = train.iloc[[]].copy()
     print("Filtered train data is empty.")
else:
    # Initialize NearestNeighbors model for filtering
    # Using 'euclidean' distance as the metric for similarity in feature space
    try:
        nn_filter = NearestNeighbors(n_neighbors=K_filter, algorithm='auto', metric='euclidean', n_jobs=-1)

        # Fit on the training features
        nn_filter.fit(X_train_features)

        # Find neighbors for each test sample
        # kneighbors returns distances and indices
        distances_filter, indices_filter = nn_filter.kneighbors(X_test_features)

        # indices_filter has shape (num_test_samples, K_filter)
        # We need to collect all unique train indices from this result
        unique_train_indices_for_filtering = np.unique(indices_filter.flatten())

        print(f"Found {len(unique_train_indices_for_filtering)} unique train images among {K_filter}-nearest neighbors for {X_test_features.shape[0]} test images.")

        # Select the corresponding rows from the *processed* train DataFrame
        # The indices in unique_train_indices_for_filtering are the row indices in X_train_features,
        # which correspond directly to the indices in train_processed_df because they were processed in order.
        filtered_train_data = train_processed_df.iloc[unique_train_indices_for_filtering].copy()

    except Exception as e:
        print(f"An error occurred during K-NN filtering: {e}")
        # Return empty filtered data in case of error
        filtered_train_data = train.iloc[[]].copy()
        print("Filtered train data is empty due to error.")


print(f"Shape of filtered_train_data: {filtered_train_data.shape}")

# --- Save Filtered Train Data CSV ---
output_filtered_train_csv = 'filtered_train_dec_jan_feb_knn50.csv'
print(f"\nSaving filtered train data to '{output_filtered_train_csv}'...")
try:
    filtered_train_data.to_csv(output_filtered_train_csv, index=False)
    print("Filtered train data saved successfully.")
except Exception as e:
    print(f"Error saving filtered train data: {e}")


# --- Generate KNN Prediction Submission ---

# Use K=1 neighbor for prediction
K_predict = 1
print(f"\nFinding {K_predict}-nearest neighbor in train for each test image to generate predictions...")

# Check if there's enough data to perform K-NN prediction
if X_train_features.shape[0] == 0 or X_test_features.shape[0] == 0 or K_predict > X_train_features.shape[0]:
    print("Cannot perform 1-NN prediction: insufficient data or K_predict is too large.")
    # Create a submission file with default predictions (e.g., majority class from original train if needed)
    # Or just predict 'Unknown' for the relevant test IDs
    submission_df = test_processed_df[['ID']].copy() # Use IDs from processed test data
    # Determine a default prediction - let's use the most frequent class from the original full train data
    if not train['Target'].empty:
         default_pred = train['Target'].mode()[0]
         print(f"Predicting default class '{default_pred}' due to missing data.")
         submission_df['Target'] = default_pred
    else:
         print("Cannot determine default class, predicting 'Unknown'.")
         submission_df['Target'] = 'Unknown'

    print("Generated submission with default predictions.")

else:
    # Initialize NearestNeighbors model for 1-NN prediction
    try:
        nn_predict = NearestNeighbors(n_neighbors=K_predict, algorithm='auto', metric='euclidean', n_jobs=-1)

        # Fit on the training features
        nn_predict.fit(X_train_features)

        # Find the single nearest neighbor for each test sample
        distances_predict, indices_predict = nn_predict.kneighbors(X_test_features)

        # indices_predict has shape (num_test_samples, 1) -> flatten to (num_test_samples,)
        one_nn_train_indices = indices_predict.flatten()

        # Get the predicted labels by looking up the Target of the 1-NN in the processed train DataFrame
        # Use iloc on the processed train_df, as indices align with X_train_features rows
        predicted_labels = train_processed_df.iloc[one_nn_train_indices]['Target'].values

        # Create the submission DataFrame
        # Use the IDs from the processed test DataFrame (which match the order of X_test_features)
        submission_df = test_processed_df[['ID']].copy()
        submission_df['Target'] = predicted_labels

        print(f"Generated {len(submission_df)} predictions based on 1-NN.")
        # print(submission_df.head()) # Optional: show head

    except Exception as e:
        print(f"An error occurred during 1-NN prediction: {e}")
        # Fallback to default prediction if an error occurs during KNN
        submission_df = test_processed_df[['ID']].copy() # Use IDs from processed test data
        if not train['Target'].empty:
             default_pred = train['Target'].mode()[0]
             print(f"Predicting default class '{default_pred}' due to error.")
             submission_df['Target'] = default_pred
        else:
             print("Cannot determine default class, predicting 'Unknown'.")
             submission_df['Target'] = 'Unknown'


# --- Save Submission File ---
output_submission_csv = 'submission_knn_jan_feb_analysis.csv'
print(f"\nSaving submission file to '{output_submission_csv}'...")
try:
    submission_df.to_csv(output_submission_csv, index=False)
    print("Submission file saved successfully.")
except Exception as e:
    print(f"Error saving submission file: {e}")

Filtering train and test data by month...
Train data for Dec, Jan, Feb: 2859 rows
Test data for Jan, Feb: 564 rows

Processing train data and collecting tiles...


Processing Tiles:   0%|          | 0/2859 [00:00<?, ?file/s]

Successfully processed 2859 tiles.

Processing test data and collecting tiles...


Processing Tiles:   0%|          | 0/564 [00:00<?, ?file/s]

Successfully processed 564 tiles.

Shape of train_processed_df: (2859, 7)
Shape of train_tiles_array: (2859, 12, 326, 326)
Shape of train_labels: (2859,)

Shape of test_processed_df: (564, 4)
Shape of test_tiles_array: (564, 12, 326, 326)
Shape of test_labels: None

Extracting features for train data...
Starting feature extraction for 2859 tiles...
Computing vegetation and other indices...
Calculating band and index statistics...


In [ ]:
#################